In [ ]:
## Synthèse  et suite ##

'''

SUITE : 
- constat 1 : les temps de séjour sont de 11 à 12 jour => inversion de tendance à Treprod = 8-9 %
- constat 2 : il y a une amorce de chute vers le 28 oct
    - effet des premiers couvres feu du 17 oct ?
    - et le temps de réaction qui est tombé à 10 jours sur le Treprod
- voir comment séparer les tendances proches de 9 % (proche inversion) et les autres (croissance)


Rappel des actions :
- couvre feu région parisienne, bouche du rhone, rhone/loire et nord le samedi 17 octobre (chiffre 10 octobre)
- couvre étendu à 54 départements le samedi 24 octobre (chiffre 17 octobre)
- reconfinement le 30 octobre



'''

print() # fin

In [ ]:
## SAUVEGARDE DES FICHIERS
# Sous Markdow : cela génère un sous repertoire avec les images
#!jupyter nbconvert --to markdown --no-input  Surveillance_covid19.ipynb
# Sous HTML
#!jupyter nbconvert --to html --no-input  Surveillance_covid19.ipynb

# Surveillance du COVID-19 en France

Bonjour,

L'objectif est de calculer les indicateurs et les modèles qui permettent de suivre l'évolution de l'épidémie de Covid-19 en France :
- les chapites I,II et III présentent les données utilisées et les principes des calculs.
- les chapitres IV et V (bilan) décrivent l'évolution de l'épidémie par département.

Une mise à jour bi-mensuelle est réalisée à partir des nouvelles données hospitalières et de tests de dépistage.

## I. Origine des données
- nombre quotidien d'hospitalisation, retour à domicile et décès par département (site data.gouv.fr)
- résultat quotidien des tests de dépistage virologiques par département (site data.gouv.fr)
- nombre d'habitant par département

In [ ]:
from matplotlib import pyplot
import matplotlib
import math
import sklearn.metrics as sm
from sklearn.cluster import KMeans
from sklearn import datasets
import numpy as np
from scipy import stats
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import datetime
# Importation des librairies pour l'analyse des composantes principales
from sklearn import datasets
from sklearn import decomposition
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import statsmodels.api as sm
## Importation des données sous forme de dataframe ##
DonneesHosp = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7',sep = ';')
DonneesDep = pd.read_csv('Donnees\Population_departement.csv',sep = ';')
DonneesHosp = DonneesHosp.dropna(how = 'any') # suppression des lignes avec NaN
DonneesTestsViro = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/406c6a23-e283-4300-9484-54e78c8ae675',sep = ';',low_memory=False)
del DonneesTestsViro['pop'] # A supprimer car les lignes avec classe d'age = 0 ont une valeur vide pour cette variable (pop)
DonneesTestsViro = DonneesTestsViro.dropna(how = 'any') # suppression des lignes avec NaN


## II. Construction des indicateurs

In [ ]:
# filtrage des lignes uniquement avec la somme homme et femme
# Lien avec les données département
Donneesmixte = DonneesHosp[DonneesHosp['sexe']==0]
DonneesTestsViro = DonneesTestsViro.loc[DonneesTestsViro['cl_age90'] == 0] # conserve somme des classes d'age
DonneesTestsViro['p'] = DonneesTestsViro['P']
DonneesTestsViro['t'] = DonneesTestsViro['T']
del DonneesTestsViro['T']
del  DonneesTestsViro['P']
# suppression colonne sexe et reanimation 
del Donneesmixte['sexe']
del Donneesmixte['rea']
Donneesmixte = Donneesmixte.drop_duplicates(subset = ['dep', 'jour'])# supprime les doublons dep+jour
Donneesmixte = Donneesmixte.reset_index(drop = True) # réindexe
for i in range(len(Donneesmixte)):
    if Donneesmixte.at[i,'jour'].find('/') > 0:
        Donneesmixte.at[i,'jour'] = (str.split(Donneesmixte.at[i,'jour'],'/')[2]+'-'
                                     +str.split(Donneesmixte.at[i,'jour'],'/')[1]+'-'
                                     +str.split(Donneesmixte.at[i,'jour'],'/')[0])
# tri les valeurs par departement et jour
Dj = Donneesmixte.sort_values(by = ['dep', 'jour']) 
# Remise en forme du numero de département sur 2 chiffres minimum (0x)
for i in range(len(DonneesDep)-1): # Parcours des lignes dans l'ordre département + jour
    if len(DonneesDep.loc[DonneesDep.index[i], 'CODDEP']) == 1:
        DonneesDep.loc[DonneesDep.index[i], 'CODDEP'] = '0' + DonneesDep.loc[DonneesDep.index[i], 'CODDEP']
# Jointure sur le département pour récupérer le nombre d'habitant (PTOT) et les tests virologiques
Dj = pd.merge(Dj, DonneesDep, left_on = ['dep'], right_on = ['CODDEP'])
Dj = pd.merge(Dj, DonneesTestsViro, left_on = ['dep','jour'], right_on = ['dep','jour'], how = 'left' )
# Suppression de colonne
del Dj['CODDEP']
del Dj['NBARR']
del Dj['NBCAN']
del Dj['PMUN']
del Dj['cl_age90']

In [ ]:
print("Calcul mis à jour le",Dj['jour'].max())

In [ ]:
# Calcul pour la France entière = somme des départements
for j in Dj[Dj['dep']== '01'].itertuples(): # itération sur tous les jours
    ligne = len(Dj) # ajoute une ligne à la fin
    Dj.loc[ligne,['dep']] = '999'
    Dj.loc[ligne,['jour']] = j.jour
    Dj.loc[ligne,['DEP']] = 'France'
    Dj.loc[ligne,['hosp']] =  Dj[Dj['jour'] == j.jour]['hosp'].sum()  
    Dj.loc[ligne,['rad']] =  Dj[Dj['jour'] == j.jour]['rad'].sum()
    Dj.loc[ligne,['dc']] =  Dj[Dj['jour'] == j.jour]['dc'].sum()
    Dj.loc[ligne,['NBCOM']] =  Dj[Dj['jour'] == j.jour]['NBCOM'].sum()
    Dj.loc[ligne,['PTOT']] =  Dj[Dj['jour'] == j.jour]['PTOT'].sum()
    if Dj[Dj['jour'] == j.jour]['p'].isna().sum() < 10:
        Dj.loc[ligne,['p']] =  Dj[Dj['jour'] == j.jour]['p'].sum()
        Dj.loc[ligne,['t']] =  Dj[Dj['jour'] == j.jour]['t'].sum()
    else:
        Dj.at[ligne,'p'] = float('nan')
        Dj.at[ligne,'p'] = float('nan')
Dj = Dj.reset_index(drop = True) # réindexe

### Calcul des indicateurs par département
- taux d'hospitalisation (Thosp) : nombre d'hospitalisation pour 100 000 habitants
- taux d'entrée à l'hôpital (Treprod) : nombre d'entrée quotidienne pour 100 hospitalisations
- taux d'entrée à l'hôpital lissé (Treprodmoy) : moyenne sur 3 jours

In [ ]:
# Initialisation de colonne de colonnnes
Dj['Entree'] = 0 # Colonne des entrées en hospitalisation
Dj['Sortie'] = 0 # Colonne des sorties en hospitalisation
Dj['Thosp'] = 0 # proportion : nb d'hospitalisation par 100 000 habitatns
Dj['Treprod'] = 0 # taux de reproduction quotidien : nb entrée  pour 100 hospitalisation
Dj['Treprodmoy'] = 0 # taux de reproduction lissé +/- 3 jours : nb entrée  pour 100 hospitalisation
Dj['tendance'] = 0 # tendance de la vitesse : pente des entrées / moyenne(entrée) sur 15 jours
Dj['incoherence'] = False # 
## Calcul des entrées, tests lissées et R0 à +/- 3 jours et ramenés à 100 000 habitants
Dj['Rj'] = float('nan') # taux de reproduction actualisé
Dj['Emoy'] = float('nan') # lissage proportion entrée +/- 3 jours
Dj['VEmoy'] = 0 # 2 si calculé avec toutes les données / 1 si calculé partiellement / 0 sinon
Dj['pmoy'] = float('nan') # lissage proportion de test +/- 3 jours
Dj['Vpmoy'] = 0 # 2 si calculé avec toutes les données / 1 si calculé partiellement / 0 sinon
Dj['Dureehosp'] = 0 # nb de jour avant une date donnée pour que la somme des entrées soit égale au nb d'hosp à cette date
Dj['Edureehosp'] = 0 # nombre d'entrée hospitalisation sur la duree hosp (calcul intermédiaire)
Dj['VThosp'] = False # Augmentation  des Thosp = True 

# Calcul entree, sortie, rapport entree/hospitalisation et tendance
dep = 'Ain'
for i in range(len(Dj)): # Parcours des lignes dans l'ordre département + jour 
    # print('\r','departement : ',i, end='')
    Dj.at[i, 'Thosp'] = round(Dj.at[i, 'hosp'] / Dj.at[i, 'PTOT'] * 100000 )
    if Dj.at[i, 'dep'] == Dj.at[i, 'dep']: 
        if Dj.at[i, 'DEP'] != dep:
            dep = Dj.at[i, 'DEP']
            print('\r','calcul entree : ' + dep + '                              ', end='')
        # Calcul des entrées, sorties et taux de reproduction
        if i > 0 and Dj.at[i, 'dep'] == Dj.at[i-1, 'dep']: # le point précédent est dans le même département 
            #if (Dj.at[i, 'rad'] > Dj.at[i-1, 'rad']) &  (Dj.at[i, 'dc'] > Dj.at[i-1, 'dc']):
            if (Dj.at[i, 'rad'] + Dj.at[i, 'dc']) > (Dj.at[i-1, 'rad'] + Dj.at[i-1, 'dc']):
                Sortie = Dj.at[i, 'rad'] - Dj.at[i-1, 'rad']  + Dj.at[i, 'dc'] - Dj.at[i-1, 'dc']
            elif (Dj.at[i, 'rad'] > Dj.at[i-1, 'rad']):
                Sortie = Dj.at[i, 'rad'] - Dj.at[i-1, 'rad']
            elif (Dj.at[i, 'dc'] > Dj.at[i-1, 'dc']):
                 Sortie =  Dj.at[i, 'dc'] - Dj.at[i-1, 'dc']
            else: Sortie = 0
            if Sortie > Dj.at[i-1, 'hosp']: # les sorties ne doivent pas dépasser les hospitalisations
                Sortie = 0
                Dj.at[i, 'incoherence'] = True
            Entree = Sortie + Dj.at[i, 'hosp'] - Dj.at[i-1, 'hosp']
            
            datedebut = datetime.date(int(str.split(Dj.at[i-1,'jour'],'-')[0]),
                                  int(str.split(Dj.at[i-1,'jour'],'-')[1]),int(str.split(Dj.at[i-1,'jour'],'-')[2]))
        
            datefin = datetime.date(int(str.split(Dj.at[i,'jour'],'-')[0]),
                                  int(str.split(Dj.at[i,'jour'],'-')[1]),int(str.split(Dj.at[i,'jour'],'-')[2]))
            Njour = (datefin - datedebut).days
            #Njour = 1
            if Entree > 0 and Entree < 0.6 * Njour * Dj.at[i, 'hosp'] and  Dj.at[i, 'hosp'] > 1:
                # en dessous de 15 hosp, le taux de reproduction a peu de sens et les entrées > 60 % du nb hosp
                Dj.at[i, 'Treprod'] = round(Entree / Njour / Dj.at[i, 'hosp'] * 100)
                Dj.at[i, 'Entree'] = Entree

            Dj.at[i, 'Sortie'] = Sortie
            
            # initialisation de la recherche de la durée d'hospitalisation (ajout d'1 jour)
            Edureehosp = Dj.at[i-1,'Edureehosp'] +  Dj.at[i, 'Entree']
            Dureehosp = Dj.at[i-1,'Dureehosp'] + 1
            
            if Edureehosp > Dj.at[i, 'hosp'] :
                while Edureehosp  > Dj.at[i, 'hosp'] :
                    Dureehosp = Dureehosp - 1
                    Edureehosp =  Edureehosp - Dj.at[i - Dureehosp, 'Entree']

                Dj.at[i,'Dureehosp'] = Dureehosp + 1
                Dj.at[i,'Edureehosp'] = Edureehosp + Dj.at[i - Dureehosp, 'Entree']
            else:
                Dj.at[i,'Dureehosp'] = Dureehosp 
                Dj.at[i,'Edureehosp'] = Edureehosp 
                
            # Calcul de l'augmentation des Thosp
            if Dj.at[i,'Thosp'] > Dj.at[i-1,'Thosp']: Dj.at[i,'VThosp'] = True
        
# lissage du nombre d'entrées et de tests positifs            
for i in range(len(Dj)): # Parcours des lignes dans l'ordre département + jour 
    # print('\r','departement : ',i, end='')
    if Dj.at[i, 'dep'] == Dj.at[i, 'dep']: 
        if Dj.at[i, 'DEP'] != dep:
            dep = Dj.at[i, 'DEP']
            print('\r','lissage entree : ' + dep + '                              ', end='')
        Emoy3 = 0
        if i>=3 and i < len(Dj)-3 and  Dj.at[i-3,'DEP'] == dep and  Dj.at[i+3,'DEP'] == dep:
            datedebut = datetime.date(int(str.split(Dj.at[i-3,'jour'],'-')[0]),
                                  int(str.split(Dj.at[i-3,'jour'],'-')[1]),int(str.split(Dj.at[i-3,'jour'],'-')[2]))
            datefin = datetime.date(int(str.split(Dj.at[i+3,'jour'],'-')[0]),
                                int(str.split(Dj.at[i+3,'jour'],'-')[1]),int(str.split(Dj.at[i+3,'jour'],'-')[2]))
            Njour = (datefin - datedebut).days
            Emoy3 =  Dj.loc[i-3:i+3,'Entree'].mean()*7/(1+Njour) 
            Dj.at[i,'Emoy'] = round(Emoy3  /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'VEmoy'] = 2
        elif i>=3 and i < len(Dj)-2 and  Dj.at[i-3,'DEP'] == dep and  Dj.at[i+2,'DEP'] == dep:
            datedebut = datetime.date(int(str.split(Dj.at[i-3,'jour'],'-')[0]),
                                  int(str.split(Dj.at[i-3,'jour'],'-')[1]),int(str.split(Dj.at[i-3,'jour'],'-')[2]))
            datefin = datetime.date(int(str.split(Dj.at[i+2,'jour'],'-')[0]),
                                int(str.split(Dj.at[i+2,'jour'],'-')[1]),int(str.split(Dj.at[i+2,'jour'],'-')[2]))
            Njour = (datefin - datedebut).days
            Emoy3 =  Dj.loc[i-3:i+2,'Entree'].mean() *6/(1+Njour)
            Dj.at[i,'Emoy'] = round(Emoy3  /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'VEmoy'] = 1
        elif i>=3 and i < len(Dj)-1 and  Dj.at[i-3,'DEP'] == dep and  Dj.at[i+1,'DEP'] == dep:
            datedebut = datetime.date(int(str.split(Dj.at[i-3,'jour'],'-')[0]),
                                  int(str.split(Dj.at[i-3,'jour'],'-')[1]),int(str.split(Dj.at[i-3,'jour'],'-')[2]))
            datefin = datetime.date(int(str.split(Dj.at[i+1,'jour'],'-')[0]),
                                int(str.split(Dj.at[i+1,'jour'],'-')[1]),int(str.split(Dj.at[i+1,'jour'],'-')[2]))
            Njour = (datefin - datedebut).days
            Emoy3 =  Dj.loc[i-3:i+1,'Entree'].mean()*5/(1+Njour)
            Dj.at[i,'Emoy'] = round(Emoy3  /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'VEmoy'] = 1
        elif i>=3 and i < len(Dj) and  Dj.at[i-3,'DEP'] == dep and  Dj.at[i,'DEP'] == dep:
            datedebut = datetime.date(int(str.split(Dj.at[i-3,'jour'],'-')[0]),
                                  int(str.split(Dj.at[i-3,'jour'],'-')[1]),int(str.split(Dj.at[i-3,'jour'],'-')[2]))
            datefin = datetime.date(int(str.split(Dj.at[i,'jour'],'-')[0]),
                                int(str.split(Dj.at[i,'jour'],'-')[1]),int(str.split(Dj.at[i,'jour'],'-')[2]))
            Njour = (datefin - datedebut).days
            Emoy3 =  Dj.loc[i-3:i,'Entree'].mean() *4/(1+Njour)
            Dj.at[i,'Emoy'] = round(Emoy3  /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'VEmoy'] = 1

        if i>=3 and i < len(Dj)-3 and  Dj.at[i-3,'DEP'] == dep and math.isnan(Dj.at[i-3,'p']) == False and  Dj.at[i+3,'DEP'] == dep and math.isnan(Dj.at[i+3,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i-3:i+3,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'Vpmoy'] = 2
        elif i>=3 and i < len(Dj)-2 and  Dj.at[i-3,'DEP'] == dep and math.isnan(Dj.at[i-3,'p']) == False and  Dj.at[i+2,'DEP'] == dep and math.isnan(Dj.at[i+2,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i-3:i+2,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'Vpmoy'] = 1
        elif i>=3 and i < len(Dj)-1 and  Dj.at[i-3,'DEP'] == dep and math.isnan(Dj.at[i-3,'p']) == False and  Dj.at[i+1,'DEP'] == dep and math.isnan(Dj.at[i+1,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i-3:i+1,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2) 
            Dj.at[i,'Vpmoy'] = 1
        elif i>=3 and i < len(Dj) and  Dj.at[i-3,'DEP'] == dep and math.isnan(Dj.at[i-3,'p']) == False and  Dj.at[i,'DEP'] == dep and math.isnan(Dj.at[i,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i-3:i,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'Vpmoy'] = 1
        # pas de valeur au début
        elif i>=3 and i < len(Dj)-3 and  Dj.at[i-2,'DEP'] == dep and math.isnan(Dj.at[i-2,'p']) == False and  Dj.at[i+3,'DEP'] == dep and math.isnan(Dj.at[i+3,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i-2:i+3,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'Vpmoy'] = 2
        elif i>=3 and i < len(Dj)-3 and  Dj.at[i-1,'DEP'] == dep and math.isnan(Dj.at[i-1,'p']) == False and  Dj.at[i+3,'DEP'] == dep and math.isnan(Dj.at[i+3,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i-1:i+3,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'Vpmoy'] = 2
        elif i>=3 and i < len(Dj)-3 and  Dj.at[i,'DEP'] == dep and math.isnan(Dj.at[i,'p']) == False and  Dj.at[i+3,'DEP'] == dep and math.isnan(Dj.at[i+3,'p']) == False:
            Dj.at[i,'pmoy'] = round(Dj.loc[i:i+3,'p'].mean()   /  Dj.at[i,'PTOT'] * 100000,2)
            Dj.at[i,'Vpmoy'] = 2
    
        if i>=14  and  Dj.at[i-14,'DEP'] == dep :
            Emoy14 = Dj.loc[i-14:i,'Emoy'].mean()
            if Emoy14 > 0:
                Dj.at[i,'Rj'] = round(Dj.at[i,'Emoy'] / Emoy14 ,2)  
            
        if Emoy3 > 0 and Dj.at[i, 'hosp'] > 15:  
            Dj.at[i, 'Treprodmoy'] = round(Emoy3 / Dj.at[i, 'hosp'] * 100) 
            
print('\r','                                                            '    )         

### Classification quotidienne des départements

In [ ]:
'''
Djtemp = Dj.loc[(Dj['DEP'] == 'Puy-de-Dôme') & (Dj['jour'] > '2020-07-10') & (Dj['jour'] < '2020-09-03'),['jour','hosp','rad','dc','Entree','Dureehosp','Edureehosp']]
pyplot.figure(figsize = (15, 5))
pyplot.plot(Djtemp['jour'], Djtemp['Dureehosp'])
pyplot.plot(Djtemp['jour'], Djtemp['Entree'])
pyplot.plot(Djtemp['jour'], Djtemp['hosp'])
for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
        label.set_rotation(90)
pyplot.grid()
Djtemp
'''
print()

In [ ]:
Dj['CThosp'] = -1 # classement du taux d'hospitalisation (en cluster)
Dj['CTreprod'] = 0 # Classement du taux de reproduction (en cluster)

## Classement quotidien des départements en taux d'hospitalisation
x = Dj[Dj['jour'] < '2020-06-15']['Thosp'] # fin phase 3 où tous les départements métropolitain sont verts
model=KMeans(n_clusters=5)
model.fit(x.values.reshape(-1, 1)) # conversion en vecteur colonne
intensite = np.zeros((4, 1), dtype = int) # limite supérieure des 4 premières zones
intensite[0] = (np.sort(model.cluster_centers_,0)[0]  + np.sort(model.cluster_centers_,0)[1]) / 2
intensite[1] = (np.sort(model.cluster_centers_,0)[1]  + np.sort(model.cluster_centers_,0)[2]) / 2
intensite[2] = (np.sort(model.cluster_centers_,0)[2]  + np.sort(model.cluster_centers_,0)[3]) / 2
intensite[3] = (np.sort(model.cluster_centers_,0)[3]  + np.sort(model.cluster_centers_,0)[4]) / 2
#for i in range(0,4):
#    print(intensite[i])
# Valeurs figées avec les données du 18 juillet (dont Mayotte)
intensite[0] = 14 # ancien 14
intensite[1] = 34
intensite[2] = 62
intensite[3] = 104


# integration des résultats dans le tableau dep+jour
for i in range(len(Dj)):
    if Dj.at[i,'Thosp'] < intensite[0]:Dj.at[i,'CThosp'] = 0
    elif Dj.at[i,'Thosp'] < intensite[1]:Dj.at[i,'CThosp'] = 1
    elif Dj.at[i,'Thosp'] < intensite[2]:Dj.at[i,'CThosp'] = 2
    elif Dj.at[i,'Thosp'] < intensite[3]:Dj.at[i,'CThosp'] = 3
    else:Dj.at[i,'CThosp'] = 4

print("Catégories pour le taux d'hospitalisation (pour 100 000 hab.) :")
print('très bas < ', intensite.reshape(-1)[0],' < bas < ', intensite.reshape(-1)[1], 
      ' < moyen < ', intensite.reshape(-1)[2], ' < élevé < ', intensite.reshape(-1)[3],' < très élevé')


## Classement quotidien des départements par taux de reproduction ##
x = Dj.loc[Dj['jour'] < '2020-05-11']['Treprodmoy'] # Début du déconfinement
model=KMeans(n_clusters=3)
model.fit(x.values.reshape(-1, 1))
# Classement des clusters dans l'ordre ascendant des proportions
intensite = np.zeros((2, 1), dtype = int) # limite supérieure des 4 premières zones
intensite[0] = (np.sort(model.cluster_centers_,0)[0]  + np.sort(model.cluster_centers_,0)[1]) / 2
intensite[1] = (np.sort(model.cluster_centers_,0)[1]  + np.sort(model.cluster_centers_,0)[2]) / 2
centreTreprod = np.zeros((3, 1), dtype = int)
centreTreprod[0] = np.sort(model.cluster_centers_,0)[0]
centreTreprod[1] = np.sort(model.cluster_centers_,0)[1]
centreTreprod[2] = np.sort(model.cluster_centers_,0)[2]

# Valeurs figées avec les données du 18 juillet (dont Mayotte)
intensite[0] = 6
intensite[1] = 16
centreTreprod[0] = 2
centreTreprod[1] = 10
centreTreprod[2] = 22
# integration des résultats dans le tableau dep+jour
for i in range(len(Dj)):
    if Dj.at[i,'Treprodmoy'] < intensite[0]:Dj.at[i,'CTreprod'] = 0
    elif Dj.at[i,'Treprodmoy'] < intensite[1]:Dj.at[i,'CTreprod'] = 1
    else:Dj.at[i,'CTreprod'] = 2
print()
print("Catégories pour le taux d'entrée à l'hôpital (pour 100 hosp.)")
print('bas < ', intensite.reshape(-1)[0], ' < élevé < ', intensite.reshape(-1)[1], ' < très élevé ')


In [ ]:
print("Pour information : classification du taux d'hospitalisation sur la période globale : ")
x = Dj[Dj['jour'] < '2020-11-07']['Thosp'] # fin phase 3 où tous les départements métropolitain sont verts
model=KMeans(n_clusters=5)
model.fit(x.values.reshape(-1, 1)) # conversion en vecteur colonne
intensitet = np.zeros((4, 1), dtype = int) # limite supérieure des 4 premières zones
intensitet[0] = (np.sort(model.cluster_centers_,0)[0]  + np.sort(model.cluster_centers_,0)[1]) / 2
intensitet[1] = (np.sort(model.cluster_centers_,0)[1]  + np.sort(model.cluster_centers_,0)[2]) / 2
intensitet[2] = (np.sort(model.cluster_centers_,0)[2]  + np.sort(model.cluster_centers_,0)[3]) / 2
intensitet[3] = (np.sort(model.cluster_centers_,0)[3]  + np.sort(model.cluster_centers_,0)[4]) / 2

print(intensitet.reshape(-1))

### Calcul d'une alerte sur le taux d'entrée à l'hôpital
Les conditions :
- le taux d'hospitalisation n'est pas très bas (> 14 hosp. / 100 000 hab.)
- le taux d'entrée à l'hôpital est élevé (> 6 entrées / 100 hosp.)

ou
- le taux d'hospitalisation est très bas (< 14 hosp. / 100 000 hab.)
- le taux d'entrée à l'hôpital est très élevé (> 16 entrées / 100 hosp)

Cette alerte détecte une hausse significative des hospitalisations (cf. IV.B.1).

In [ ]:
Dj['alerte'] = False # indique un taux d'entré élevé
Dj['alerteconf'] = False # indique une alerte sur la semaine précédent
Dj['alertePLSR'] = 0
## Creation d'une alerte si sur les 4 derniers jours le taux de reproduction lissé n'est pas faible

datemax = datetime.date(int(str.split(Dj['jour'].max(),'-')[0]),
                        int(str.split(Dj['jour'].max(),'-')[1]),
                        int(str.split(Dj['jour'].max(),'-')[2]))

Nbjour14 = datetime.timedelta(days = 14); datemin14 = datemax - Nbjour14;Sdatemin14 = str(datemin14)

jmax = Dj['jour'].max();
dep = 'Ain'
for i in range(len(Dj)): # Parcours des lignes dans l'ordre département + jour 
 #   if ((Dj.at[i,'jour'] >= '2019-05-01' and Dj.at[i,'jour'] <= '2021-06-15') 
 #       or (Dj.at[i,'jour'] >= '2020-09-01')) : # phase 3
    if Dj.at[i, 'DEP'] != dep:
        dep = Dj.at[i, 'DEP']
        print('\r','calcul alarme : ' + dep + '                              ', end='')
    if Dj.at[i,'CThosp'] == 0  and Dj.at[i,'CTreprod'] < 2 :Dj.at[i,'alerte'] = False
    elif Dj.at[i,'CThosp'] == 0  and Dj.at[i,'CTreprod'] == 2:Dj.at[i,'alerte'] = True
    elif (Dj.at[i,'CThosp'] > 0 ) and Dj.at[i,'CTreprod'] == 0:Dj.at[i,'alerte'] = False
    elif (Dj.at[i,'CThosp'] > 0 ) and Dj.at[i,'CTreprod'] > 0 :Dj.at[i,'alerte'] = True
    if Dj.at[i,'jour'] == jmax:
        if (Dj.at[i-3,'alerte'] == True or Dj.at[i-2,'alerte'] == True 
            or Dj.at[i-1,'alerte'] == True or Dj.at[i,'alerte'] == True): 
            if (Dj.at[i-10,'alerte'] == True or Dj.at[i-9,'alerte'] == True 
                or Dj.at[i-8,'alerte'] == True or Dj.at[i-7,'alerte'] == True):
                Dj.at[i,'alerteconf'] = True
print('\r','                                                                                   ', end='')

# Alerte confirmé manuelle
Dj.loc[(Dj['DEP'] == 'Hérault') & (Dj['jour'] >= '2020-10-14') & (Dj['jour'] <= '2020-10-21'),'alerteconf'] = True  
Dj.loc[(Dj['DEP'] == 'Isère') & (Dj['jour'] >= '2020-10-14') &(Dj['jour'] <= '2020-10-21'),'alerteconf'] = True
Dj.loc[(Dj['DEP'] == 'Puy-de-Dôme') & (Dj['jour'] >= '2020-10-16') &(Dj['jour'] <= '2020-10-23'),'alerteconf'] = True
Dj.loc[(Dj['DEP'] == 'Seine-Maritime') & (Dj['jour'] >= '2020-10-16') &(Dj['jour'] <= '2020-10-23'),'alerteconf'] = True
Dj.loc[(Dj['DEP'] == 'Pas-de-Calais') & (Dj['jour'] >= '2020-10-16') &(Dj['jour'] <= '2020-10-23'),'alerteconf'] = True
Dj.loc[(Dj['DEP'] == 'France') & (Dj['jour'] >= '2020-10-16') &(Dj['jour'] <= '2020-10-23'),'alerteconf'] = True

Dj.loc[(Dj['DEP'] == 'Haute-Garonne') & (Dj['jour'] >= '2020-10-19') &(Dj['jour'] <= '2020-10-26'),'alerteconf'] = True

In [ ]:
Dj.loc[(Dj['DEP'] == 'France') & (Dj['jour'] >= '2020-10-16') &(Dj['jour'] <= '2020-10-23'),'alerteconf'] = True

### Construction des courbes (première partie)

In [ ]:
###  Evolution des taux d'hospitalisation

In [ ]:
# Visualisation du taux d'hospitalisation pour l'ensemble des départements ##
def courbe_TxHosp(dep):
    #fig1 = pyplot.figure(1,figsize = (15, 5))
    pyplot.figure(figsize = (15, 5))
    #for i in range(len(Dj)):
    lDEP = Dj['DEP'].drop_duplicates().values
    for DEP in lDEP:
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
        if (Dj.at[i,'jour'] == Dj['jour'].max()) and (
            (dep == 'France') or (Dj.at[i,'DEP'] == dep ) or (Dj.at[i,'DEP'] == 'France' )) : # recherche de la dernière ligne du département
        # Choix des couleurs, épaisseur en fonction du taux hosp, taux reprod et tendance
            epaisseur = 1
            legende =''
            transparence = 0.5
            if Dj.at[i,'CThosp'] >= 3 :
                if  Dj.loc[i-3:i,'alerte'].max() == True:col = 'red';epaisseur = 3;transparence =1
                elif Dj.loc[i-3:i,'PredicTest'].max() >= 20:col = 'yellow';epaisseur = 2;transparence = 0.7
                else:col = 'darkgoldenrod';epaisseur = 3;transparence =1
            elif Dj.at[i,'CThosp'] >= 1 :
                if  Dj.loc[i-3:i,'alerte'].max() == True :col = 'deeppink';epaisseur = 3;transparence =1
                elif Dj.loc[i-3:i,'PredicTest'].max() >= 20:col = 'yellow';epaisseur = 2;transparence = 0.7
                else:col = 'darkorange';epaisseur = 1;transparence =1    
            elif Dj.loc[i-3:i,'alerte'].max() == True :col = 'deeppink';epaisseur = 3;transparence =1
            elif Dj.loc[i-3:i,'PredicTest'].max() >= 20:col = 'yellow';epaisseur = 3;transparence = 0.7
            else:col ='grey';transparence =0.7
            if Dj.at[i,'dep'] == '999':epaisseur = 5 ;col = 'black';transparence =1
            elif Dj.at[i,'DEP'] == dep:epaisseur = 5 ;col = 'blue';legende =Dj.at[i,'DEP'];transparence =1
        # Courbe du département       
            Djdep = Dj.loc[Dj['dep'] == Dj.at[i,'dep'],['jour','Thosp']] # récupération du département
            pyplot.plot(Djdep['jour'],Djdep['Thosp'],col,linewidth = epaisseur,label = legende, alpha = transparence)
    # Paramètres généraux
    if dep == 'France':
        pyplot.plot(0,0,'red', linewidth = 5,          label = "critique")
        pyplot.plot(0,0,'darkgoldenrod',linewidth = 5, label = "élevé")
        pyplot.plot(0,0,'darkorange',linewidth = 3,    label = "moyen")
        pyplot.plot(0,0,'deeppink',linewidth = 5, label = "Alerte sur les entrées à l'hôpital")
        pyplot.plot(0,0,'yellow',linewidth = 5, label = "Alerte sur les tests positifs")
        #pyplot.plot(0,0,'grey',linewidth = 1,label = "taux d'hosp. faible")
    pyplot.plot(0,0,'black',linewidth = 5, label = 'France')
    pyplot.legend(loc='upper center')
    pyplot.title("Evolution du taux d'hospitalisation par département")
    pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
    pyplot.ylabel('hosp. / 100 000 hab. ' , fontsize = 12) # titre des ordonnées
    axes = pyplot.gca()
    #axes.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(30)) # impose 8 graduations maxi
    axes.xaxis.axes.set_xticks([0,14, 44,75, 105,136,167,197,228,258])
    axes.xaxis.axes.set_xticks([29,59,90,120,151,182,212,243,273],minor = True)
    axes.xaxis.grid(True, which = 'minor')
    for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
        label.set_rotation(90)
    pyplot.grid() # grille
    pyplot.show()
    #if dep == 'France':fig1.savefig('Images\Evolution des hospitalisations_' + dep + '.png')
#courbe_TxHosp('France')

In [ ]:
### Evolution des taux de reproduction

In [ ]:
## Visualisation du taux d'hospitalisation de l'ensemble des départements pour vérifer la cohérence ##
def courbe_TxEntree(dep):
#dep = '63'
    pyplot.figure(figsize = (15, 5))
    #for i in range(len(Dj)):
    lDEP = Dj['DEP'].drop_duplicates().values
    for DEP in lDEP:
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()    
        if (Dj.at[i,'jour'] == Dj['jour'].max()) and (
            (dep == 'France') or (Dj.at[i,'DEP'] == dep ) or (Dj.at[i,'DEP'] == 'France' )): 
            # recherche de la dernière ligne du département
            couleur = 'grey'; epaisseur = 1; legende ='';transparence = 0.7
            if  Dj.at[i,'dep'] == '999' : couleur = 'black' ; epaisseur = 5 ; legende='France';transparence = 1
            elif  Dj.at[i,'DEP'] == dep : couleur = 'blue' ; epaisseur = 5 ; legende=Dj.at[i,'DEP'];transparence = 1
            elif Dj.loc[i-3:i,'alerte'].max() == True : couleur = 'deeppink';epaisseur = 2;transparence = 1
            elif Dj.loc[i-3:i,'PredicTest'].max() >= 20:couleur = 'gold';epaisseur = 2;transparence = 0.7
            # Courbe du département
            if Dj.at[i, 'incoherence'] == False:
                Djdep = Dj.loc[Dj['dep'] == Dj.at[i,'dep'],['jour','Treprodmoy']] # récupération du département
                pyplot.plot(Djdep['jour'],Djdep['Treprodmoy'],
                            couleur,linewidth=epaisseur,label =legende, alpha = transparence)
    # Paramètres généraux
    pyplot.axhline(y = centreTreprod [2],color ='red',linewidth = 3)
    if centreTreprod [1] != 0 :
        pyplot.axhline(y = centreTreprod [1],color ='deeppink',linewidth = 3)
    pyplot.axhline(y = centreTreprod [0],color ='grey',linewidth = 3)
    pyplot.title("Evolution du taux d'entrée à l'hôpital par département")
    pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
    pyplot.ylabel("Entrée / 100 hosp." , fontsize = 12) # titre des ordonnées
    axes = pyplot.gca()
    #axes.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(30)) # impose 8 graduations maxi
    axes.xaxis.axes.set_xticks([0,14, 44,75, 105,136,167,197,228,258])
    axes.xaxis.axes.set_xticks([29,59,90,120,151,182,212,243,273],minor = True)
    axes.xaxis.grid(True, which = 'minor')
    for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
        label.set_rotation(90)
    pyplot.grid() # grille
    if dep == 'France':
        pyplot.plot(0,0,'deeppink',linewidth = 5, label = "Alerte sur les entrées à l'hôpital")
        pyplot.plot(0,0,'gold',linewidth = 5, label = "Alerte sur les tests positifs")
    pyplot.legend(loc='upper right')
    pyplot.show() 
    #if dep == 'France':fig2.savefig('Images\Evolution des entrées_' + dep + '.png')
#courbe_TxEntree('France')

In [ ]:
### Evolution pour les taux d'hospitalisation les plus élevés

In [ ]:
# Visualisation du taux d'hospitalisation pour l'ensemble des départements ##
def courbe_TxHospHaut(dep):
    icol = 0
    cm = matplotlib.cm.get_cmap('tab20')
    pyplot.figure(figsize = (15, 5))
    #for i in range(len(Dj)):
    lDEP = Dj['DEP'].drop_duplicates().values
    for DEP in lDEP:
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
        # recherche de la catégorie maxi des départements pour les afficher
        if (Dj.at[i,'jour'] == Dj['jour'].max()) and ( 
            (Dj.at[i,'DEP'] == 'France') |
            (Dj.at[i,'DEP'] == dep) |
            (Dj.at[i,'CThosp'] == 4) | 
            ((Dj.loc[ (Dj['jour'] == Dj['jour'].max()),'CThosp'].max() == 4) and (Dj.at[i,'CThosp'] == 3)) |
            ((Dj.loc[ (Dj['jour'] == Dj['jour'].max()),'CThosp'].max() == 3) and (Dj.at[i,'CThosp'] == 3)) |
            ((Dj.loc[ (Dj['jour'] == Dj['jour'].max()),'CThosp'].max() == 2) and (Dj.at[i,'CThosp'] == 2)) |
            ((Dj.loc[ (Dj['jour'] == Dj['jour'].max()),'CThosp'].max() == 1) and (Dj.at[i,'CThosp'] == 1)) ):
        # Choix des couleurs, épaisseur en fonction du taux hosp, taux reprod et tendance
            legende =  Dj.at[i,'DEP']
            if icol >= 20:icol = 0
            col = cm.colors[icol]
            icol = icol + 1 
            epaisseur = 5
            if Dj.at[i,'DEP'] == 'France':epaisseur = 5 ;col = 'black'
            elif Dj.at[i,'DEP'] == dep:epaisseur = 5 ;col = 'grey'
        # Courbe du département       
            Djdep = Dj.loc[Dj['dep'] == Dj.at[i,'dep'],['jour','Thosp']] # récupération du département
            pyplot.plot(Djdep['jour'],Djdep['Thosp'],c=col,linewidth = epaisseur,label = legende)
    # Paramètres généraux
    pyplot.legend(loc='upper center')
    pyplot.title("Evolution des départements avec les taux d'hospitalisation les plus élevés")
    pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
    pyplot.ylabel('hosp. / 100 000 hab. ' , fontsize = 12) # titre des ordonnées
    axes = pyplot.gca()
    #axes.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(30)) # impose 8 graduations maxi
    axes.xaxis.axes.set_xticks([0,14, 44,75, 105,136,167,197,228,258])
    axes.xaxis.axes.set_xticks([29,59,90,120,151,182,212,243],minor = True)
    axes.xaxis.grid(True, which = 'minor')
    for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
        label.set_rotation(90)
    pyplot.grid() # grille
    pyplot.show()
    
#courbe_TxHospHaut('France') # pour test

In [ ]:
## Construction des courbes R0 et suivi des tests virologiques

In [ ]:
## Evolution du R0 et des entrées lisées dans un département sélectionné
def R0_departement(dep):
    ## Courbe
    fig5 = pyplot.figure(5,figsize = (15, 5))
    # Taux de reproduction
    axe1=pyplot.gca()
    axe1.plot(Dj[(Dj['DEP'] == dep) ]['jour'],
              Dj[(Dj['DEP'] == dep) ]['Emoy'],c='blue',linewidth = 5,linestyle = ' ')
    axe1.plot(Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 2)]['jour'],
              Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 2)]['Emoy'],c='blue',linewidth = 5,label='Entree',linestyle = '-')
    axe1.plot(Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 1)]['jour'],
              Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 1)]['Emoy'],c='blue',linewidth = 3,label='En cours',linestyle = '--')
    axe1.legend( loc='upper left') # 
    pyplot.yscale('log')
    pyplot.ylabel('Entree / 100 000 hab')
    # Délimitation des phases
    axe1.vlines(x = '2020-03-18',color ='red',linewidth = 3,linestyle='--',ymin = 0,ymax= 5.0)
    axe1.vlines(x = '2020-05-11',color ='deeppink',linewidth = 3,linestyle='--',ymin = 0, ymax= 5.0)
    axe1.vlines(x = '2020-06-02',color ='deeppink',linewidth = 3,linestyle='--',ymin = 0, ymax= 5.0)
    axe1.vlines(x = '2020-06-15',color ='green',linewidth = 3,linestyle='--',ymin = 0, ymax= 5.0)
    axe1.vlines(x = '2020-07-20',color ='gold',linewidth = 3,linestyle='--',ymin = 0, ymax= 5.0)
    axe1.text('2020-03-18', 0.04, ' Confinement ', horizontalalignment = 'left',
                verticalalignment = 'center',color='red',fontsize = 12,fontweight = 'bold')
    axe1.text('2020-05-11', 0.04, ' Deconfinement ', horizontalalignment = 'right',
                verticalalignment = 'center',color='deeppink',fontsize = 12,fontweight = 'bold')
    axe1.text('2020-05-11', 0.04, ' < 100 km ', horizontalalignment = 'left',
                verticalalignment = 'center',color='deeppink',fontsize = 12,fontweight = 'bold')
    axe1.text('2020-06-02', 0.04, ' Restaurant ', horizontalalignment = 'left',
                verticalalignment = 'center',color='deeppink',fontsize = 12,fontweight = 'bold')
    axe1.text('2020-06-15', 0.02, ' Loisirs ', horizontalalignment = 'left',
                verticalalignment = 'center',color='green',fontsize = 12,fontweight = 'bold')
    axe1.text('2020-07-20', 0.04, ' Masque lieux clos ', horizontalalignment = 'left',
                verticalalignment = 'center',color='gold',fontsize = 12,fontweight = 'bold')

    # axes des entrées
    axe2 = pyplot.gca().twinx()
    Ylim = np.ones((len(Dj[(Dj['DEP'] == dep) ]), 1), dtype = float)
    axe2.plot(Dj[(Dj['DEP'] == dep) ]['jour'],
              Ylim,c='red',
                    linewidth = 3, label = 'critique',linestyle = '--')
    axe2.plot(Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 2)]['jour'],
              Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 2)]['Rj'],c='black',
                    linewidth = 3, label = 'R0',linestyle = '-')
    axe2.plot(Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 1)]['jour'],
              Dj[(Dj['DEP'] == dep) & (Dj['VEmoy'] == 1)]['Rj'],c='black',
                    linewidth = 3, label = 'en cours',linestyle = '--')
    axe2.legend( loc='upper right') # grille
    pyplot.ylabel('R0')
    pyplot.ylim(0, 2)
    #axe2.text(Dj.loc[(Dj['jour'] == Dj['jour'].max()) & (Dj['DEP'] == dep), 'jour'],
    #            Dj.loc[(Dj['jour'] == Dj['jour'].max()) & (Dj['DEP'] == dep), 'Rj'],
    #             " "+ str(Dj.loc[(Dj['jour'] == Dj['jour'].max()) & (Dj['DEP'] == dep), 'Rj'].values[0]),
    #            horizontalalignment = 'left',verticalalignment = 'center',color='black',fontsize = 12,fontweight = 'bold')

    axe1.set_yticks([0.01,0.02,0.05,0.1,0.2,0.5,1,2,5,10])
    axe1.yaxis.set_ticklabels(['0.01','0.02','0.05','0.1','0.2','0.5','1','2','5', '10'])
    # Paramètres courbe
    for label in axe1.xaxis.get_ticklabels():
        label.set_rotation(90)
    axe1.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(30)) # impose 8 graduations maxi
    axe2.grid() # grille
    pyplot.title("Evolution du taux de reproduction et des entrées : " + dep)
    pyplot.xlabel('Date' , fontsize = 12) # titre des absisses

    pyplot.show()

#R0_departement('France')

### III. Relation entre les entrées à l'hôpital et les résultats des tests virologiques.

### Modélisation à partir d'une régression logistique PLS

Le modèle établit la relation entre les entrées à l'hôpital et les tests positifs pour :
- anticiper une hausse des entrées à l'hôpital,
- vérifier que les entrées à l'hôpital sont accompagnées de tests de dépistage.

La relation a été établie pendant la phase 1 et 2 du déconfinement (jusqu'au 15 juin) où les tests de dépistage sont mis en place.

**Les données explicatives** sont les taux de test positif (/ 100 000 hab.) sur les 10 derniers jours précédent l'alerte.

**Les données à prédire** sont une hausse significative des hospitalisation (alerte lorsque les entrées quotidiennes dépassent  6 entrées pour 100 hospitalisations)).

In [ ]:
## régression logistique 
# Observation sur y(dep,j) = taux de reproduction élevée en hospital au jour  j pour le departement dep 
# variable explicative : test positif (j-i, dep) pour i = 0 à 9 (sur les 10 derniers jours)

## Structuration des données
dep = 'France'
npoint = 10 # nombre d'historique à prendre en compte
Yentree = np.zeros((len(Dj), 1), dtype = float)
Xtest = np.zeros((len(Dj), npoint), dtype = float)
Index = np.zeros((len(Dj),1), dtype = int)
j = 0
for i in range(len(Dj)): # Parcours des lignes dans l'ordre département + jour
    if  Dj.at[i,'jour'] < '2020-06-15':# période d'apprentissage : phase 1 et 2
        if math.isnan(Dj.at[i,'CTreprod']) == False and  math.isnan(Dj.at[i,'pmoy']) == False:
            if (i >= (npoint - 1) and Dj.at[i,'dep'] == Dj.at[i- (npoint - 1),'dep'] 
                and math.isnan(Dj.at[i-(npoint - 1),'pmoy']) == False and 
                (Dj.loc[i,'alerte'] == False or Dj.loc[i-1,'alerte'] == False 
                or Dj.loc[i-2,'alerte'] == False or Dj.loc[i-3,'alerte'] == False)) :
                    # on ne prend plus en compte  au delà de 3 alertes consécutives
                    #print(Dj.loc[i,['DEP','jour']].values)
                    if Dj.loc[i,'alerte'] == True : 
                        Yentree[j] = 1
                        #print(Dj.loc[i,['DEP','jour']].values)
                    Xtest[j,:] = Dj.loc[i-(npoint - 1):i ,'pmoy'].values
                    for k in range((npoint - 1)): # remplacement des valeurs NaN par la valeur précédente
                        if math.isnan(Xtest[j,k+1]) == True:
                            Xtest[j,k+1] = Xtest[j,k] # 

                    Index[j] = i # pour retrouver la ligne d'origine
                    j = j + 1
Xtest = Xtest[0:j,:] # redimensionne
Index = Index[0:j]
YCTreprod = Yentree[0:j]

## Calcul du la regression logistique PLS de l'alerte des entrées sur les tests virologiques
# Initialisation
Xm = Xtest.mean(0) # moyenne par variables
X = Xtest - Xm # matrice des tests centrées
Y = YCTreprod
k = len(X[1,:]) # nombre de variables explicatives
n = len(X[:,1]) # nombre d'observation

In [ ]:
## régression logistique  des entrées hospitalières sur les tests positifs
# Observation sur y(dep,j) =  entrées hospitalières élevés (> 4 pour 100 000 hab.)
# variable explicative : test positif (j-i, dep) pour i = 10 (sur les dernières semaines)


## Structuration des données
dep = 'France'
npoint = 10 # nombre d'historique à prendre en compte
Yentree = np.zeros((len(Dj), 1), dtype = float)
Xtest = np.zeros((len(Dj), npoint), dtype = float)
Index = np.zeros((len(Dj),1), dtype = int)
j = 0
for i in range(len(Dj)): # Parcours des lignes dans l'ordre département + jour
    if  Dj.at[i,'jour'] < '2020-06-15':# période d'apprentissage : phase 1 et 2
        if math.isnan(Dj.at[i,'CTreprod']) == False and  math.isnan(Dj.at[i,'pmoy']) == False:
            if (i >= (npoint - 1) and Dj.at[i,'dep'] == Dj.at[i- (npoint - 1),'dep'] 
                and math.isnan(Dj.at[i-(npoint - 1),'pmoy']) == False and 
                (Dj.loc[i,'alerte'] == False or Dj.loc[i-1,'alerte'] == False 
                or Dj.loc[i-2,'alerte'] == False or Dj.loc[i-3,'alerte'] == False)) :
                    # on ne prend plus en compte  au delà de 3 alertes consécutives
                    #print(Dj.loc[i,['DEP','jour']].values)
                    if Dj.loc[i,'alerte'] == True : 
                        Yentree[j] = 1
                        #print(Dj.loc[i,['DEP','jour']].values)
                    Xtest[j,:] = Dj.loc[i-(npoint - 1):i ,'pmoy'].values
                    for k in range((npoint - 1)): # remplacement des valeurs NaN par la valeur précédente
                        if math.isnan(Xtest[j,k+1]) == True:
                            Xtest[j,k+1] = Xtest[j,k] # 

                    Index[j] = i # pour retrouver la ligne d'origine
                    j = j + 1
Xtest = Xtest[0:j,:] # redimensionne
Index = Index[0:j]
YCTreprod = Yentree[0:j]

## Calcul du la regression logistique PLS de l'alerte des entrées sur les tests virologiques
# Initialisation
Xm = Xtest.mean(0) # moyenne par variables
X = Xtest - Xm # matrice des tests centrées
Y = YCTreprod
k = len(X[1,:]) # nombre de variables explicatives
n = len(X[:,1]) # nombre d'observation

In [ ]:
# 1ère composante

w1 = np.zeros(k, dtype = float)
t1 = np.zeros(n,dtype = float)
# RLog
for j in range(k):
    Xj = X[:,j] # variables Xj
    Xj_stat = sm.add_constant(Xj)
    model = sm.Logit(Y, Xj_stat)
    result = model.fit(disp=0)
    w1[j] = result.params[1]
w1=1/(np.linalg.norm(w1))*w1 # normalisation
t1 = np.dot(X,w1)
w1f = w1.reshape(1,-1)
# RLog sur t1
t1_stat = sm.add_constant(t1)
model = sm.Logit(Y, t1_stat)
result = model.fit(disp=0)
# Calcul du résidu X1 par RegLin
regressor = LinearRegression()
regressor.fit(t1.reshape(-1, 1), X)
p1 = regressor.coef_ 
X1 =  X - np.dot(t1.reshape(-1,1),p1.reshape(1,-1)) # résidu de X
result1 = result
## Affichage des résultats
"""
print()
print('1ère composante insuffisante')
print()
print ('Matrice de confusion pour une probabilité à plus de 20 %')
print('ligne : valeur 0/1 et colonne : prédiction 0/1')
print(result.pred_table(0.2))
"""
print()

In [ ]:
# 2ème composante

w2 = np.zeros(k, dtype = float)
t2 = np.zeros(n,dtype = float)
for j in range(k):
    tX1j = np.concatenate([t1.reshape(-1,1), X1[:,j].reshape(-1,1)], axis = 1)
    tX1j_stat = sm.add_constant(tX1j)
    model = sm.Logit(Y, tX1j_stat)
    result = model.fit(disp=0)
    w2[j] = result.params[2]
w2=1/(np.linalg.norm(w2))*w2 # normalisation
t2 = np.dot(X1,w2)
# Calcul de w2* qui relie t2 à X et non X1
w2f=w2-w1*np.dot(w2.reshape(1,-1),p1.reshape(-1,1))
w2ftest = np.dot(np.identity(k)-np.dot(w1.reshape(-1,1),p1.reshape(1,-1)),w2) # plus propre

# RLog sur t1,t2
tx = np.concatenate([t1.reshape(-1,1),t2.reshape(-1,1)], axis = 1)
tx_stat = sm.add_constant(tx)
model = sm.Logit(Y, tx_stat)
result = model.fit(disp=0)
result2 = result
# Calcul du résidu X2 par RegLin
regressor = LinearRegression()
regressor.fit(t2.reshape(-1, 1), X1)
p2 = regressor.coef_ 
X2 =  X1 - np.dot(t2.reshape(-1,1),p2.reshape(1,-1)) # résidu de X

## Affichage des résultats
"""
print()
print('2ème composante insuffisante')
print()
print ('Matrice de confusion pour une probabilité à plus de 20 %')
print('ligne : valeur 0/1 et colonne : prédiction 0/1')
print(result.pred_table(0.2)) 
"""
print()

In [ ]:
# 3ème composante

# calcul t2,w2 par RegLog
w3 = np.zeros(k, dtype = float)
t3 = np.zeros(n,dtype = float)
for j in range(k):
    tX2j = np.concatenate([t1.reshape(-1,1),t2.reshape(-1,1), X2[:,j].reshape(-1,1)], axis = 1)
    tX2j_stat = sm.add_constant(tX2j)
    model = sm.Logit(Y, tX2j_stat)
    result = model.fit(disp=0)
    w3[j] = result.params[3]
w3=1/(np.linalg.norm(w3))*w3 # normalisation
t3 = np.dot(X2,w3)
# Calcul de w3* qui relie t3 à X et non X2
w3f1=w3-w2*np.dot(w3.reshape(1,-1),p2.reshape(-1,1)) # t3 = X1*w3f1
w3f = w3f1-w1*np.dot(w3f1.reshape(1,-1),p1.reshape(-1,1)) # t3 = X*w3f

# RLog sur t1,t2,t3
tx = np.concatenate([t1.reshape(-1,1),t2.reshape(-1,1),t3.reshape(-1,1)], axis = 1)
tx_stat = sm.add_constant(tx)
model = sm.Logit(Y, tx_stat)
result = model.fit(disp=0)
result3 = result
tx3 = tx_stat # matrice des variables latentes t par observations + 1 constante
# Calcul du résidu X3 par RegLin
regressor = LinearRegression()
regressor.fit(t3.reshape(-1, 1), X2)
p3 = regressor.coef_ 
X3 =  X2 - np.dot(t3.reshape(-1,1),p3.reshape(1,-1)) # résidu de X

## Affichage des résultats

'''
print('3ème composante')


print(result3.summary()) # test significatif à 5 % (LLR p-value pour le modèle global / P>z pour un coefficient)

print()
print('w1f : ' ,np.round(w1f.reshape(1,-1),1),'p1 : ' ,np.round(p1.reshape(1,-1),1))
print('w2f : ' ,np.round(w2f.reshape(1,-1),1),'p2 : ' ,np.round(p2.reshape(1,-1),1))
print('w3f : ' ,np.round(w3f.reshape(1,-1),1),'p3 : ' ,np.round(p3.reshape(1,-1),1))
print()
print ('Matrice de confusion pour une probabilité à plus de 20 %')
print('ligne : valeur 0/1 et colonne : prédiction 0/1')
print(np.round(result.pred_table(0.2),0)) 

pyplot.figure(12,figsize = (15, 10))
pyplot.subplot(3,2,1) # prédiction
pyplot.plot(100 * result.predict(tx_stat),Y,linestyle='',marker='.')
pyplot.xlim(0,100)
pyplot.xlabel("%")
pyplot.ylabel('Alerte sur les entrées = 1')
pyplot.title("Probabilité d'avoir une alerte sur les entrées / tests positifs")
pyplot.subplot(3,2,2) # composantes
pyplot.title('courbe des composantes')
pyplot.plot(Xm,label='moy')
pyplot.plot(p1,label='p1')
pyplot.plot(p2,label='p2')
pyplot.plot(p3,label='p3')
pyplot.xlabel('jour')
pyplot.grid()
pyplot.legend()
pyplot.subplot(3,2,3) # cartes des charges 1 et 2
pyplot.xlim(1.2*min(result.params[1],0,w1f.min()),1.2*max(result.params[1],w1f.max()))
pyplot.ylim(1.2*min(result.params[3],w3f.min(),result.params[2],w2f.min()),
            1.2*max(result.params[3],w3f.max(),result.params[2],w2f.max()))
for j in range(len(w1f[0,:])):
    pyplot.annotate('X'+str(j),(w1f[0,j],w2f[0,j]),color='red')
pyplot.annotate('Y12',(result.params[1],result.params[2]),color='blue')
pyplot.grid()

pyplot.subplot(3,2,4) # carte des charges 1 et 3
pyplot.xlim(min(1.2*result.params[1],0,w1f.min()),1.2*max(result.params[1],w1f.max()))
pyplot.ylim(1.2*min(result.params[3],w3f.min(),result.params[2],w2f.min()),
            1.2*max(result.params[3],w3f.max(),result.params[2],w2f.max()))
for j in range(len(w1f[0,:])):
    pyplot.annotate('X'+str(j),(w1f[0,j],w3f[0,j]),color='red')
pyplot.annotate('Y13',(result.params[1],result.params[3]),color='blue')
pyplot.grid()
pyplot.subplot(3,2,5)
# Impact des variables / moyennes
pyplot.plot((result.params[1]*w1f+result.params[2]*w2f+result.params[3]*w3f).reshape(-1,1))
pyplot.grid()
# impact récent : on enclenche des tests si entrées importantes
# impact à 7 jours : les cas testés entrent 7 jours plus tard à l'hopital (durée incubation/symptomes)

#print(result3.pred_table(0.2)) 

print()
#w1f = [[0.3 0.3 0.3 0.2 0.2 0.2 0.3 0.3 0.4 0.5]]
#w2f = [[ 0.1  0.  -0.  -0.2 -0.3 -0.4 -0.2  0.2  0.6  0.6]]
#w3f = [[ 0.1  0.2  0.5  0.4 -0.4 -0.6 -0.3 -0.2  0.1  0.5]]
#p1 = [[0.3 0.3 0.3 0.4 0.4 0.4 0.4 0.4 0.3 0.3]].reshape(-1,1)
#p2 = [[-0.3 -0.4 -0.5 -0.4 -0.4 -0.2 -0.   0.2  0.5  0.6]].reshape(-1,1)
#p3 = [[ 0.9  0.7  0.4  0.1 -0.2 -0.4 -0.5 -0.5 -0.3 -0.2]].reshape(-1,1)
#xm = [[-5.72,0.35,0.83,0.81]]
'''
print()

In [ ]:
# 4ème composante

# calcul t4,w4 par RegLog
w4 = np.zeros(k, dtype = float)
t4 = np.zeros(n,dtype = float)
for j in range(k):
    tX3j = np.concatenate([t1.reshape(-1,1),t2.reshape(-1,1),t3.reshape(-1,1), X3[:,j].reshape(-1,1)], axis = 1)
    tX3j_stat = sm.add_constant(tX3j)
    model = sm.Logit(Y, tX3j_stat)
    result = model.fit(disp=0)
    w4[j] = result.params[4]
w4=1/(np.linalg.norm(w4))*w4 # normalisation
t4 = np.dot(X3,w4)
# Calcul de w2* qui relie t2 à X et non X1
#w4f2=np.dot(1-np.dot(w3.reshape(-1,1),p3.reshape(1,-1)),w4.reshape(-1,1))
#w4f1=np.dot(1-np.dot(w2.reshape(-1,1),p2.reshape(1,-1)),w4f2.reshape(-1,1))
#w4f=np.dot(1-np.dot(w1.reshape(-1,1),p1.reshape(1,-1)),w4f1.reshape(-1,1))

w4f2=w4-w3*np.dot(w4.reshape(1,-1),p3.reshape(-1,1)) # t4 = X2*w3f2
w4f1 = w4f2-w2*np.dot(w4f2.reshape(1,-1),p2.reshape(-1,1)) # t4 = X1*w3f1
w4f = w4f1-w1*np.dot(w4f1.reshape(1,-1),p1.reshape(-1,1)) # t4 = X*w3f

# RLog sur t1,t2,t3
tx = np.concatenate([t1.reshape(-1,1),t2.reshape(-1,1),t3.reshape(-1,1),t4.reshape(-1,1)], axis = 1)
tx_stat = sm.add_constant(tx)
model = sm.Logit(Y, tx_stat)
result = model.fit(disp=0)
result4 = result
# Calcul du résidu X3 par RegLin
regressor = LinearRegression()
regressor.fit(t4.reshape(-1, 1), X3)
p4 = regressor.coef_ 
X4 =  X3 - np.dot(t4.reshape(-1,1),p4.reshape(1,-1)) # résidu de X

## Affichage des résultats
'''
print()
print('4ème composante inutile ')
print()
print ('Matrice de confusion pour une probabilité à plus de 20 %')
print('ligne : valeur 0/1 et colonne : prédiction 0/1')
print(result.pred_table(0.2)) 
'''
print()

In [ ]:
'''
### Interprétation du modèle

La modélisation montre que les alertes sur le taux d'entrée à l'hôpital sont liées à 3 facteurs : 
- la moyenne du taux de tests positifs sur les 10 derniers jours,
- la pente pendant ces 10 jours,
- l'allure exponentielle de la croissance.
'''
print()

In [ ]:
'''
pyplot.title('courbe des composantes')
pyplot.plot(Xm,label='moy')
pyplot.plot(p1,label='p1')
pyplot.plot(p2,label='p2')
pyplot.plot(p3,label='p3')
pyplot.xlabel('jour')
pyplot.grid()
pyplot.legend()
pyplot.show()
'''
print()


In [ ]:
'''
#L'exemple** ci-dessous de la Mayenne au 27 juin illustre cette décomposition :
- la courbe BLEUE est l'évolution réelle des tests positifs,
- en ORANGE : la tendance moyenne,
- en VERT : ajout de l'évolution à la hausse,
- en ROUGE : ajout de la courbure (exponentielle) de la tendance pour représenter l'évolution prise en compte par le modèle.

Cela traduit que l'évolution des tests positifs de la Mayenne au 27 juin suit une croissance exponentielle.
'''
print()

In [ ]:
'''
dep = 'Mayenne'
jour ='2020-06-27'
xnew = Dj.loc[(Dj['DEP'] == dep) & (Dj['jour'] <= jour ) ,'pmoy'].iloc[-npoint:].values
xnew = xnew - Xm
txnew = [1,np.dot(xnew,w1f.reshape(-1,1))[0],np.dot(xnew,w2f.reshape(-1,1))[0],np.dot(xnew,w3f.reshape(-1,1))[0]]
print()
pyplot.plot((Xm+txnew[1] * p1.reshape(-1)),color='orange',label ='courbe moyenne',linewidth=1)
pyplot.plot((Xm+txnew[1] * p1.reshape(-1)+txnew[2] * p2.reshape(-1))
            ,color='green',label = 'moyenne + tendance',linewidth=2)
pyplot.plot((Xm+txnew[1] * p1.reshape(-1)+txnew[2] * p2.reshape(-1)+txnew[3] * p3.reshape(-1))
            ,color='red',label = 'moyenne + tendance + courbure',linewidth=3)
pyplot.plot(Xm +xnew,color ='blue',linewidth=5,label='courbe réelle')
pyplot.ylim(0,5)
pyplot.xlabel("Jour précédent")
pyplot.ylabel("Nombre de test positifs")
pyplot.title ('Tests positifs de la Mayenne sur les 10 jours avant le 27 juin')
pyplot.legend(loc='upper left')
pyplot.grid()
'''
print()

In [ ]:
'''
Le tableau suivant présente **les coefficients de la régression** x1, x2 et x3 pour chacune des composantes qui ont un poids équivalent.

Les résultats aux tests statistiques de significativité du modèle globale et des différents coefficients montrent que  les alertes sur les entrées hospitalières sont en relation avec les tests de dépistage positifs des 10 derniers jours.
'''
print()

In [ ]:
#print(result3.summary()) # test significatif à 5 % (LLR p-value pour le modèle global / P>z pour un coefficient)

In [ ]:
'''
### Analyse de la précision du modèle
Les alertes réelles sur les entrées hospitalières et celles prédites par le modèle à partir des tests positifs  ont été comparées :
'''
print()

In [ ]:
'''
print("Alertes détectées par le modèle    :", int(result3.pred_table(0.2)[1,1])) 
print()
print("Alertes non détectées              :", int(result3.pred_table(0.2)[1,0])) 
print()
print("Cas sans alerte                    :", int(result3.pred_table(0.2)[0,0])) 
print()
print("Fausses alertes vues par le modèle  :", int(result3.pred_table(0.2)[0,1]))
'''
print()

In [ ]:
# Analyse des fausses alertes et alertes non détectées dans la période d'apprentissage
Dj['Explication'] =''
Dj['AlerteTest'] = 'SA' # Déclaration des alertes sur les tests virologiques
#

DEP = 'Allier';
Jour = '2020-05-22';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : test peu élevé / entrée' 

DEP = 'Creuse';
Jour = '2020-05-29';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : test peu élevé / entrée'

DEP = 'Meuse';
Jour = '2020-06-04';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : fin alerte en J+1'

DEP = 'Yonne';
Jour = '2020-06-09';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : Données rassemblées'
Jour = '2020-06-10';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : Données rassemblées'
Jour = '2020-06-11';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : Données rassemblées'

DEP = 'Guyane';
Jour = '2020-05-23';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : tests récents/entrées'
Jour = '2020-05-24';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : tests récents/entrées'
Jour = '2020-05-25';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : tests récents/entrées'
Jour = '2020-05-29';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : rechute ponctuelle des tests'
Jour = '2020-05-30';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : rechute ponctuelle des tests'
Jour = '2020-05-31';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='ND : rechute ponctuelle des tests'

DEP = 'Mayotte';
Jour = '2020-06-02';Dj.loc[(Dj['DEP'] == DEP) & (Dj['jour'] == Jour),'Explication'] ='justifié : cf. J+1 '


#parcours des données dont le test n'est pas vérifié à 20 % alors qu'il y a des tests positifs
#print()
#print("Analyse de la pertinence du modèle sur la période d'apprentissage (avant 15 juin)")
#print()
#print('Département non détecté avec des tests significatifs')
for j in range(len(Index)):
    if Yentree[j] == 1 and Xtest[j,:].max() >= 2.5 and result3.predict(tx3)[j] < 0.2:
        #print()
        #print( j, '|' ,round(result3.params[1]*tx3[j,1],1),round(result3.params[2]*tx3[j,2],1),
        #     round(result3.params[3]*tx3[j,3],1),
        #      '| proba :' , round(result3.predict(tx3)[j],2),' | Max :', round(Xtest[j,:].max(),1),
        #       '|',Dj.loc[Index[j],'jour'].values[0],Dj.loc[Index[j],'DEP'].values[0],
        #      '|', Dj.loc[Index[j],'Explication'].values[0])
        Dj.loc[Index[j],'AlerteTest'] = 'ND'
print()
#print('Département non détecté avec peu de tests significatifs')
for j in range(len(Index)):
    if Yentree[j] == 1 and  result3.predict(tx3)[j] < 0.2 and Xtest[j,:].max() < 2.5 :
        #print()
        #print( j, '|' ,round(result3.params[1]*tx3[j,1],1),round(result3.params[2]*tx3[j,2],1),
        #      round(result3.params[3]*tx3[j,3],1),
        #      '| proba :' , round(result3.predict(tx3)[j],2),' | Max :', round(Xtest[j,:].max(),1),
        #       '|',Dj.loc[Index[j],'jour'].values[0],Dj.loc[Index[j],'DEP'].values[0],
        #      '|', Dj.loc[Index[j],'Explication'].values[0])
        
        Dj.loc[Index[j],'AlerteTest'] = 'ND'
print()
#print('Fausses alertes')
for j in range(len(Index)):
    if Yentree[j] == 0 and  result3.predict(tx3)[j] > 0.2  :
        #print()
        #print( j, '|' ,round(result3.params[1]*tx3[j,1],1),round(result3.params[2]*tx3[j,2],1),
        #      round(result3.params[3]*tx3[j,3],1),
        #      '| proba :' , round(result3.predict(tx3)[j],2),' | Max :', round(Xtest[j,:].max(),1),
        #       '|',Dj.loc[Index[j],'jour'].values[0],Dj.loc[Index[j],'DEP'].values[0],
        #      '|', Dj.loc[Index[j],'Explication'].values[0])
        
        Dj.loc[Index[j],'AlerteTest'] = 'AlF'
print()        
#print('alertes detectées')
for j in range(len(Index)):
    if Yentree[j] == 1 and  result3.predict(tx3)[j] >= 0.2  :
       # print()
        #print( j, '|' ,round(result3.params[1]*tx3[j,1],1),round(result3.params[2]*tx3[j,2],1),
        #      round(result3.params[3]*tx3[j,3],1),
        #      '| proba :' , round(result3.predict(tx3)[j],2),' | Max :', round(Xtest[j,:].max(),1),
        #       '|',Dj.loc[Index[j],'jour'].values[0],Dj.loc[Index[j],'DEP'].values[0])
        
        Dj.loc[Index[j],'AlerteTest'] = 'AlV'
        Dj.loc[Index[j],'Explication'] = 'OK'

In [ ]:
'''
**Les alertes non détectées**   par le modèle  s'expliquent :
- pour l'Allier et la Creuse, les tests positifs ont été faibles par rapport aux entrées à l'hôpital : l'intérêt du modèle est donc de détecter les départements qui n'utilisent pas suffisamment les tests de dépistage,
- pour la Guyane, le nombre de tests positfs encore faible (qui sera ensuite suivi par d'une croissance des tests positifs détectée par le modèle) traduit une tendance début juin encore incertaine.
'''
print()

In [ ]:
# Alertes sur les entrées non détectées ou accompagnées par des tests positifs
#courbe_alerte_test(Dj[(Dj['jour'] < '2020-06-15') & (Dj['AlerteTest'] == 'ND')]['DEP'].drop_duplicates().values)

In [ ]:
'''
**Les fausses alertes** détectées par le modèle  s'expliquent :
- La Marne et  la Meurthe-et-Moselle ont une hausse des tests positifs début juin (> 2 cas pour 100 000 habitants pendant 1 semaine) alors que les taux d'hospitalisation sont encore élevés : pour des départements encore sous tension, une telle alerte est donc pertinente,
- pour la Meuse et la Guyane, les alertes anticipent la hausse des entrées à l'hôpital,
- l'alerte sur les tests positifs de la Vienne est en relation avec des entrées qui augmentent (même si le taux d'hospitalisation est très faible, ce qui explique l'absence d'alerte sur le taux d'entrée à l'hôpital).
'''
print()

In [ ]:
'''
### Précision du modèle
Nous constatons que :
- Les alertes sur le taux d'entrée à l'hôpital sont en lien avec les tests positifs sur les 10 derniers jours.
- Lorsque le modèle n'explique par ces alertes, les tests de depistage ont été insuffisants.
- Lorsque le modèle propose des alertes basées sur les tests positifs sans une hausse des entrées hospitalières, celles-ci anticipent une évolution à la hausse.

Ce modèle permet donc d'anticiper une éventuelle accélération de la circulation du virus dans un département.
'''
print()

In [ ]:
# Prédiction du modèle (département+date) après la période d'apprentissage (15 juin)
Dj['PredicTest'] = 0.0
Dj['AlerteTest'] = 'NaN'
datemax = datetime.date(int(str.split(Dj['jour'].max(),'-')[0]),
                        int(str.split(Dj['jour'].max(),'-')[1]),
                        int(str.split(Dj['jour'].max(),'-')[2]))

Nbjour14 = datetime.timedelta(days = 21); datemin14 = datemax - Nbjour14;Sdatemin14 = str(datemin14)
lDep = []
for i in range(len(Dj)):
    if Dj.at[i,'jour'] > Sdatemin14: # phase 3
        xnew = Dj.loc[(Dj['DEP'] == Dj.at[i,'DEP']) & (Dj['jour'] <= Dj.at[i,'jour']) ,'pmoy'].iloc[-npoint:].values
        xnew = xnew - Xm
        txnew = [1,np.dot(xnew,w1f.reshape(-1,1))[0],np.dot(xnew,w2f.reshape(-1,1))[0],np.dot(xnew,w3f.reshape(-1,1))[0]]
        if round(result3.predict(txnew)[0],2) >= 0.2 and Dj.at[i,'alerte'] == False:
            #print(Dj.at[i,'DEP'],Dj.at[i,'jour'],'Alerte sur les tests')
            Dj.at[i,'AlerteTest'] = 'AlF'
        elif round(result3.predict(txnew)[0],2) >= 0.2 and Dj.at[i,'alerte'] == True:
            #print(Dj.at[i,'DEP'],Dj.at[i,'jour'],'Alerte sur les entrées et les tests')
            Dj.at[i,'AlerteTest'] = 'AlV'
        elif  round(result3.predict(txnew)[0],2) < 0.2 and Dj.at[i,'alerte'] == True:
            #print(Dj.at[i,'DEP'],Dj.at[i,'jour'],'Alerte sur les entrées non accompagnée de test')
            Dj.at[i,'AlerteTest'] = 'ND'
            #if not(Dj.at[i,'DEP'] in lDep):# liste de département pour affichage
            #    lDep.append( Dj.at[i,'DEP'])
        elif  round(result3.predict(txnew)[0],2) < 0.2 and Dj.at[i,'alerte'] == False:
            Dj.at[i,'AlerteTest'] = 'False'
        else: Dj.at[i,'AlerteTest'] = 'NaN'
        Dj.at[i,'PredicTest'] = round(result3.predict(txnew)[0]*100)
# calcul de la date - 1 semaine
Nbjour = datetime.timedelta(days = 7); datemin = datemax - Nbjour;Sdatemin = str(datemin)
dateTVlast = Dj.loc[(Dj['Vpmoy'] == 1)] ['jour'].max() # dernière date avec une valeurs sur les tests positifs
#Sdatemin = '2020-06-15' # pour la première analyse
#print('Début de la période de surveillance : ', Sdatemin)

### Construction de graphiques (deuxième partie)

In [ ]:
# Courbe de décomposition et d'impact des tests positifs pour un département à une date donnée

def Courbe_decomposition(departement,jour):
    dep = departement
    xnew = Dj.loc[(Dj['DEP'] == dep) & (Dj['jour'] <= jour ) ,'pmoy'].iloc[-npoint:].values
    xnew = xnew - Xm
    txnew = [1,np.dot(xnew,w1f.reshape(-1,1))[0],np.dot(xnew,w2f.reshape(-1,1))[0],np.dot(xnew,w3f.reshape(-1,1))[0]]
    print()
    print('Décomposition RL/PLS pour le département :',dep,' le ',jour)
    print()
    #print("Poids des composantes                   : ",np.round(txnew[1:],1))
    #print("Coefficient de la Regression logistique : ",np.round(result3.params[1:],1))
    print('Impact des composantes                  : ',np.round(result3.params[0:]*txnew[0:],1))
    if np.isnan(result3.predict(txnew)[0]) == False:
        print('Probabilité  de croissance              : ', int(round(100 * result3.predict(txnew)[0])),'%')
    pyplot.plot((Xm+txnew[1] * p1.reshape(-1)),color='orange',label ='moyenne',linewidth=1)
    pyplot.plot((Xm+txnew[1] * p1.reshape(-1)+txnew[2] * p2.reshape(-1))
                ,color='green',label = '+ tendance',linewidth=2)
    pyplot.plot((Xm+txnew[1] * p1.reshape(-1)+txnew[2] * p2.reshape(-1)+txnew[3] * p3.reshape(-1))
                ,color='red',label = '+ courbure',linewidth=3)
    pyplot.plot(Xm +xnew,color ='blue',linewidth =5,label ='courbe réelle')
    pyplot.ylim(0,max(5,1.2*np.max(xnew)))
    pyplot.legend (loc ='upper left')
    pyplot.grid()
    pyplot.show()
    

In [ ]:
## Evolution des tests virologiques positifs pour les départements (si France) dont la proba d'une alerte > seuil donnée
def courbe_TxTestPositif():
    pyplot.figure(figsize = (15, 5))
    #for i in range(len(Dj)):
        
    lDEP = Dj['DEP'].drop_duplicates().values
    for DEP in lDEP:
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
        if (Dj.at[i,'jour'] == Dj['jour'].max()): # recherche de la dernière ligne du département

            if Dj.at[i,'DEP'] == 'France' :couleur = 'black';epaisseur = 3;transparence = 1
            elif Dj.loc[i-3:i,'alerte'].max() == True :couleur = 'deeppink';epaisseur = 3;transparence = 1
            elif Dj.loc[i-3:i,'PredicTest'].max() >= 20:couleur = 'gold';epaisseur = 3;transparence = 0.7
            else:couleur = 'grey';epaisseur = 1;transparence = 0.7

            pyplot.plot(Dj.loc[Dj['dep'] == Dj.at[i,'dep']].iloc[-91:]['jour'],
                        Dj.loc[Dj['dep'] == Dj.at[i,'dep']].iloc[-91:]['pmoy'],
                        c=couleur,linewidth = epaisseur,linestyle='-',alpha = transparence) 

    # Paramètres généraux
    pyplot.axhline(y = 3.5,color ='red',linewidth = 2,linestyle='--')
    pyplot.plot(0,0,'deeppink',linewidth = 5, label = "Alerte sur les entrées à l'hôpital")
    pyplot.plot(0,0,'gold',linewidth = 5, label = 'Alerte sur les tests positifs')
    pyplot.plot(0,0,'black',linewidth = 5, label = 'France')
    pyplot.legend(loc='upper left')
    pyplot.title("Evolution du taux de tests positifs par département")
    pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
    pyplot.ylabel('Tests positifs / 100 000 hab. ' , fontsize = 12) # titre des ordonnées
    #pyplot.ylim(0,16)
    axes = pyplot.gca()
    #axes.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(30)) # impose nb graduations maxi
    axes.xaxis.axes.set_xticks([4, 18,32,46,60,74,81,88])
    axes.xaxis.axes.set_xticks([11,25,39,53,67],minor = True)
    axes.xaxis.grid(True, which = 'minor')
    for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
        label.set_rotation(90)
    pyplot.yscale('log')
    axes.set_yticks([0.1,0.2,0.5,1,2,5,10,20,50,100,200,500])
    axes.yaxis.set_ticklabels(['0.1','0.2','0.5','1','2','5', '10','20','50','100','200','500'])
    minmax = axes.xaxis.get_view_interval()
   # pyplot.ylim(0.1, minmax[1])
    pyplot.grid() # grille
    pyplot.show()
    

#courbe_TxTestPositif()

In [ ]:
## Affichage pour un département (à gauche : hopitalisation /  à droite : tests positifs)
def courbe_departement(ldep):
    pyplot.figure(figsize = (15, max(5,5.0*len(ldep))))
    for idep in range(len(ldep)):
        dep = ldep[idep]
        # Courbe taux d'hospitalisation et d'entrée à l'hôpital
        pyplot.subplot(len(ldep),2,2*idep+1)
        if dep == 'France': couleur = 'black'
        else: couleur = 'blue' ;
        pyplot.plot(Dj[Dj['DEP'] == dep]['jour'],Dj[Dj['DEP'] == dep]['Thosp'],couleur,linewidth = 5,label='Hospitalisation pour 100 000 habitants')
        #pyplot.plot(Dj[Dj['DEP'] == dep]['jour'],Dj[Dj['DEP'] == dep]['Treprod'],'grey',linewidth = 1,label="Entrée pour 100 hospitalisations")
        pyplot.plot(Dj[Dj['DEP'] == dep]['jour'],Dj[Dj['DEP'] == dep]['Treprodmoy'],'grey',linewidth = 5,label="Entrée lissée sur 7 jours")
        if dep != 'France':
            pyplot.plot(Dj[Dj['DEP'] == 'France']['jour'],Dj[Dj['DEP'] == 'France']['Thosp'],'black',linewidth = 5 ,label='Hospitalisation France')
            pyplot.plot(Dj[Dj['DEP'] == 'France']['jour'],Dj[Dj['DEP'] == 'France']['Treprodmoy'],'black',linewidth = 3, label='Entrée France')
        
        axes = pyplot.gca()
        axes.yaxis.grid()
        axes.xaxis.grid()
        #axes.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(8)) # impose 8 graduations maxi
        axes.xaxis.axes.set_xticks([0, 44, 105,167,229])
        axes.xaxis.axes.set_xticks([14,75,136,197,259],minor = True) 
        axes.xaxis.grid(True, which = 'minor')
        axes.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(6)) # impose 8 graduations maxi
        axes.xaxis.set_tick_params(labelsize = 8)
              
        pyplot.title(Dj[Dj['DEP'] == dep]['DEP'].max(),color ='blue',fontweight = 'bold' )
        '''
        if centreTreprod [1] != 0 :
            pyplot.axhline(y = (centreTreprod [1] + centreTreprod [1])/2,color ='deeppink',linewidth = 2,linestyle='--')
        else:
            pyplot.axhline(y = (centreTreprod [2] + centreTreprod [0])/2,color ='deeppink',linewidth = 2,linestyle='--')
        '''
        pyplot.axhline(y = 8,color ='deeppink',linewidth = 2,linestyle='--') # limite Tx reprod quotidien
        # Affichage de toutes les alertes sur les tests
        '''
        if Dj.loc[(Dj['DEP'] == dep) & (Dj['AlerteTest'] != 'SA') & (Dj['AlerteTest'] != 'NaN') & (Dj['AlerteTest'] != 'False') ]['jour'].nunique() != 0:
            jmax = Dj.loc[(Dj['DEP'] == dep) & (Dj['AlerteTest'] != 'SA') & (Dj['AlerteTest'] != 'NaN')& (Dj['AlerteTest'] != 'False') ]['jour'].values
            jal = Dj.loc[(Dj['DEP'] == dep) & (Dj['AlerteTest'] != 'SA') & (Dj['AlerteTest'] != 'NaN') & (Dj['AlerteTest'] != 'False')]['AlerteTest'].values
            #print(jmax)
            for ijmax in range(len(jmax)):
                if jal[ijmax] == 'AlV': couleur = 'red'
                if jal[ijmax] == 'AlF': couleur = 'yellow' 
                if jal[ijmax] == 'ND': couleur = 'orange' 
                pyplot.plot(jmax[ijmax],Dj.loc[(Dj['DEP'] == dep) & (Dj['jour'] == jmax[ijmax] ) ]['Treprodmoy'],
                    linewidth = 10,color=couleur,linestyle='',marker='*')
        '''
        # Courbe taux de tests positifs et  d'entrées à l'hôpital        
        pyplot.subplot(len(ldep),2,2*idep+2)        
        Djdep = Dj.loc[(Dj['DEP'] == dep),['jour','pmoy','Emoy','VEmoy','Vpmoy','AlerteTest']].iloc[-43:]
        pyplot.plot(Djdep['jour'],Djdep['Emoy'],c='blue',linewidth = 5,linestyle ='')
        pyplot.plot(Djdep['jour'],Djdep['pmoy'],c='grey',linewidth = 3,linestyle ='')

        Djdep2 = Djdep.loc[(Djdep['VEmoy'] == 2)] # récupération du département
        pyplot.plot(Djdep2['jour'],Djdep2['Emoy'],c='blue',linewidth = 5,label = "Entrées",linestyle ='-')
        Djdep2 = Djdep.loc[(Djdep['Vpmoy'] == 2)]
        pyplot.plot(Djdep2['jour'],Djdep2['pmoy'],c='grey',linewidth = 3,label = "tests positif",linestyle ='-')

        Djdep2 = Djdep.loc[(Djdep['VEmoy'] == 1)] # récupération du département
        pyplot.plot(Djdep2['jour'],Djdep2['Emoy'],c='blue',linewidth = 5,linestyle ='--')
        Djdep2 = Djdep.loc[(Djdep['Vpmoy'] == 1)] # récupération du département
        pyplot.plot(Djdep2['jour'],Djdep2['pmoy'],c='grey',linewidth = 3,linestyle ='--')

        # France
        Djdepf = Dj.loc[(Dj['DEP'] == 'France'),['jour','pmoy','Emoy','VEmoy','Vpmoy']].iloc[-43:] # récupération du département
        pyplot.plot(Djdepf['jour'],Djdepf['Emoy'],c='blue',linewidth = 5,linestyle ='')
        pyplot.plot(Djdepf['jour'],Djdepf['pmoy'],c='grey',linewidth = 3,linestyle ='')

        Djdep2 = Djdepf.loc[(Djdepf['VEmoy'] == 2)] # récupération du département
        pyplot.plot(Djdep2['jour'],Djdep2['Emoy'],c='black',linewidth = 5,label = "Entrées France",linestyle ='-')
        Djdep2 = Djdepf.loc[(Djdepf['Vpmoy'] == 2)]
        pyplot.plot(Djdep2['jour'],Djdep2['pmoy'],c='black',linewidth = 2,label = "tests positif France",linestyle ='-')

        Djdep2 = Djdepf.loc[(Djdepf['VEmoy'] == 1)] # récupération du département
        pyplot.plot(Djdep2['jour'],Djdep2['Emoy'],c='black',linewidth = 5,linestyle ='--')
        Djdep2 = Djdepf.loc[(Djdepf['Vpmoy'] == 1)] # récupération du département
        pyplot.plot(Djdep2['jour'],Djdep2['pmoy'],c='black',linewidth = 2,linestyle ='--')
      
        '''        
        if Djdep.loc[ (Djdep['AlerteTest'] != 'SA')  & (Djdep['AlerteTest'] != 'False') &(Djdep['AlerteTest'] != 'NaN')]['jour'].nunique() != 0:
            jmax = Djdep.loc[ (Djdep['AlerteTest'] != 'SA')& (Djdep['AlerteTest'] != 'False') &(Djdep['AlerteTest'] != 'NaN') ]['jour'].values
            jal = Djdep.loc[(Djdep['AlerteTest'] != 'SA') & (Djdep['AlerteTest'] != 'False') &(Djdep['AlerteTest'] != 'NaN')]['AlerteTest'].values
            for ijmax in range(len(jmax)):
                if jal[ijmax] == 'AlV': couleur = 'red'
                if jal[ijmax] == 'AlF': couleur = 'yellow' 
                if jal[ijmax] == 'ND': couleur = 'orange' 
                #print(jmax[ijmax],Djdep.loc[(Dj['jour'] == jmax[ijmax] ) ]['pmoy'])
                #print (jmax)
                pyplot.plot(jmax[ijmax],Djdep.loc[(Dj['jour'] == jmax[ijmax] ) ]['pmoy'],
                    linewidth = 10,color=couleur,linestyle='',marker='*')
        '''        
        pyplot.axhline(y = 4,color ='red',linewidth = 2,linestyle='--')
        
        pyplot.yscale('log')
        axe1=pyplot.gca()
        axe1.set_yticks([0.05,0.1,0.2,0.5,1,2,5,10,20,50,100,200,500])
        axe1.yaxis.set_ticklabels(['0.05','0.1','0.2','0.5','1','2','5', '10','20','50','100','200','500'])

        #axe1.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(6)) # impose 8 graduations maxi
        axe1.xaxis.axes.set_xticks([0, 7, 14,21,28,35,42])
        axe1.xaxis.set_tick_params(labelsize = 8)

        pyplot.ylim(ymin = 0.05)
        
        pyplot.grid() # grille       
                        
    pyplot.show()  
# tests
#courbe_departement(['Paris','Ain'])
#courbe_alerte_test(Dj[(Dj['jour'] > Sdatemin) & (Dj['AlerteTest'] == 'AlV')]['DEP'].drop_duplicates().values)

In [ ]:
# Identification des départements en alerte sur la carte de France (fonction carte_alerte)

#from LatLon import string2latlon
from LatLon23 import string2latlon
import time
import threading
import random

ps, codes = [], []

def locDept(d):# coordonné pour un département
    for i in range(len(ps)):
        if str(codes[i])==str(d):
            return float(ps[i].lon), float(ps[i].lat)
    return -1.,-1.

def getVirusMap():# positionne sur les département
    lons,lats, couleurs, sizes,deps =[],[],[],[],[]
    #for i in range(len(Dj)):
    lDEP = Dj['DEP'].drop_duplicates().values
    for DEP in lDEP:
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
        if Dj.at[i,'jour'] == Dj['jour'].max():
            lon,lat = locDept(Dj.at[i,'dep'])
            #print(Dj.at[i,'DEP'],Dj.at[i,'dep'], round(lon,1), round(lat,1))
            #print(lon,lat)
            #if lon>=0 and lat>=0 :
            if lat>=0 :
                if Dj.at[i,'CThosp'] >= 3 :
                    lo = lon-0.0;la = lat+0.0
                    lons.append(lo);lats.append(la)
                    sizes.append((Dj.at[i,'CThosp'])*(Dj.at[i,'CThosp'])*50)
                    deps.append(Dj.at[i,'DEP'] )
                    if Dj.at[i,'alertePLSR'] == 2 :couleurs.append('red')
                    elif Dj.at[i,'alertePLSR'] == 1 :couleurs.append('darkorange')
                    else:couleurs.append('yellow')
                elif Dj.at[i,'CThosp'] >= 1 :
                    lo = lon-0.0;la = lat+0.0
                    lons.append(lo);lats.append(la)
                    sizes.append(Dj.at[i,'CThosp']*Dj.at[i,'CThosp']*50)
                   
                    if Dj.at[i,'alertePLSR'] == 2 :couleurs.append('red');deps.append(Dj.at[i,'DEP'] )
                    elif Dj.at[i,'alertePLSR'] == 1 :couleurs.append('darkorange');deps.append(Dj.at[i,'DEP'] )
                    else:couleurs.append('grey');deps.append('' )
                else:
                    lo = lon-0.0;la = lat+0.0
                    lons.append(lo);lats.append(la)
                    sizes.append(50)
                    deps.append(Dj.at[i,'DEP'] )
                    if Dj.at[i,'alertePLSR'] == 2 :couleurs.append('red')
                    elif Dj.at[i,'alertePLSR'] == 1 :couleurs.append('darkorange')
                    else:couleurs.append('green')
    return lons, lats, couleurs, sizes, deps

# initialisation

ps, codes = [], []

ligne = 1
# récupération des centres de département
with open("Donnees\centres_dept.txt","r", encoding='utf-8') as f:
    for l in f:
        s = l.split("\t")
        slat, slon = s[1], s[2]
        slat+=" N"
        slon=slon.replace('O',' W')
        slon=slon.replace('  ',' ')
        p = string2latlon(slat,slon,'d%°%m%’%S%" %H')
        ps.append(p)
        if ligne == 1 :codes.append('01') # pour l'Ain où il ajoute sinon à gauche \efeff (???)
        elif ligne == 20:codes.append('2A') # pour la Corse 
        elif ligne == 21:codes.append('2B') # pour la Corse
        else:codes.append(s[0])
        ligne = ligne + 1
   
    lons = [float(p.lon) for p in ps] # centre département
    lats = [float(p.lat) for p in ps]

BBox = (-5.3, 8.6, 42.4, 51.4) # limite de la carte

def carte_alerte():
    img = pyplot.imread('Donnees\France.png')
    fig, ax = pyplot.subplots(figsize = (15,15))

    dscat = ax.scatter(lons, lats, zorder=1, alpha= 0.9, c='black', s=2)# Affiche les centres de département

    # Affichage des centres des départements en fonction de la criticité
    tlons,tlats, tcouleurs, tsizes, tdeps = getVirusMap()
    scat = ax.scatter(tlons, tlats, zorder=1, alpha= 0.9, c=tcouleurs, s=tsizes)
    

    # Affichage du nom du département
    for i in range(len(tdeps)):
        ax.text(tlons[i]+0.2, tlats[i], tdeps[i], 
                horizontalalignment = 'left', verticalalignment = 'center',fontweight = 'bold')
        
    ax.add_artist(matplotlib.patches.Rectangle((-5.3, 49.7), 3.6, 1.7, 
                                               zorder = 0.2, alpha = 1,facecolor = 'white',edgecolor = 'black'))
    
    ax.scatter(-5, 51, zorder=1, alpha= 1, c='red', s=450)
    ax.scatter(-5, 51, zorder=2, alpha= 1, c='darkorange', s=200)
    ax.text(-5+0.4, 51, "Entrées critiques/élevées", horizontalalignment = 'left', verticalalignment = 'center')
    
    
    ax.scatter(-5, 50.50, zorder=1, alpha= 1, c='yellow', s=200)
    ax.text(-5+0.4, 50.50, "Hosp. élevées", horizontalalignment = 'left', verticalalignment = 'center')

    ax.scatter(-5, 50, zorder=1, alpha= 1, c='green', s=50)
    ax.text(-5+0.4, 50, "Hosp. et entrées faibles", horizontalalignment = 'left', verticalalignment = 'center')

    ax.set_title("Alertes sur les entrées hospitalières du " + Dj['jour'].max())
    ax.set_xlim(BBox[0],BBox[1])
    ax.set_ylim(BBox[2],BBox[3])
    ax.imshow(img, zorder=0, extent = BBox, aspect= 'equal')

    pyplot.show()
    
    
    fig.savefig('Images\Alertes_entrees_' + Dj['jour'].max() + '.png')
    fig.savefig('Images\Alertes_entrees_recentes' + '.png')
    
    ###('https://github.com/smarcovici/Covid_19/tree/master/Surveillance_deconfinement/Images')
    #pyplot.savefig('Alertes sur les entrees_'  + '.png')           
               
#carte_alerte()

In [ ]:
# Courbe de la croissance des entrées hospitalières pour un département

regressor = LinearRegression();regressorfin = LinearRegression();regressorbrut = LinearRegression()

njour = 24
lnE = np.zeros(njour, dtype = float)
Ebrut3 = np.zeros(njour, dtype = float)
TlnE = np.zeros((njour,1),dtype = float)
for i in range(len(TlnE)):TlnE[i,0] = i
pf = 1

def croissance_exp(DEPcible):
    pf = 1
    njour = 24
    for DEP in Dj['DEP'].drop_duplicates().values:
        
        lnE = Dj.loc[Dj['DEP'] == DEP]['Emoy'].iloc[-njour:].values.reshape(-1, 1)
        Ebrut = Dj.loc[Dj['DEP'] == DEP]['Entree'].iloc[-njour:].values.reshape(-1)
        Ebrut = Ebrut * 100000 / Dj.loc[Dj['DEP'] == DEP]['PTOT'].iloc[-1:].values.reshape(-1)
          
        for i in range(len(lnE)):
            if lnE[i] > 0: lnE[i] = np.maximum(np.log(lnE[i]),-5)
            else:lnE[i] = - 5
                
        regressor.fit(TlnE[-njour:-3], lnE[-njour:-3])
        kE = math.exp(7*regressor.coef_[0,0]) # augmentation sur 1 semaine
        R2 = regressor.score(TlnE[-njour:-3], lnE[-njour:-3])
 
        regressorfin.fit(TlnE[-10:-3], lnE[-10:-3])
        kEfin = math.exp(7*regressorfin.coef_[0,0]) # augmentation sur la dernière semaine
        
        model = sm.OLS(lnE[-njour:-3], sm.add_constant(TlnE[-njour:-3]))
        results = model.fit()
        DW = sm.stats.stattools.durbin_watson(results.resid)
        DWbrut = sm.stats.stattools.durbin_watson(Ebrut[-njour:-3]-np.exp(results.fittedvalues))
       
        divergence = False
        if (abs((kE-1)*100) >= 45 or abs((kEfin-1)*100) >=45) :
            if (((kEfin-1) / (kE-1)) > 1.5 or  ((kEfin-1) / (kE-1)) < 0.7):divergence = True
        elif abs((kEfin-1)*100 - (kE-1)*100) > 15:divergence = True
                                       
        if ((DEPcible == 'Tous'  
                 and ((R2 >= 0.7) or (abs(DW-2) <= 0.7) or (abs(DWbrut-2) <= 0.7))
                 and divergence == False) or DEP == DEPcible):

        
            for j in range(len(Ebrut3)):
                if j <= 2 :Ebrut3[j] = np.mean(Ebrut[0:3])
                elif j<=6 :Ebrut3[j] = np.mean(Ebrut[3:7])
                elif j<=9 :Ebrut3[j] = np.mean(Ebrut[7:10])
                elif j<=13 :Ebrut3[j] = np.mean(Ebrut[10:14])
                elif j<=16 :Ebrut3[j] = np.mean(Ebrut[14:17])    
                elif j<=20 :Ebrut3[j] = np.mean(Ebrut[17:21])
                elif j<=23 :Ebrut3[j] = np.mean(Ebrut[21:24])
            
            if DEPcible != 'Tous':
                pyplot.figure(pf,figsize = (12, 4))
            else: 
                pyplot.figure(pf,figsize = (3, 1))

            pyplot.rcParams.update({'figure.max_open_warning': 0})
            pyplot.title(DEP)
            pyplot.plot(TlnE,np.exp(lnE),'blue')
            pyplot.plot(TlnE[-njour:-3],np.exp(results.fittedvalues),'red',linewidth = 5)
            pyplot.plot(TlnE,Ebrut,'black')
            pyplot.plot(TlnE,Ebrut3,'blue',linewidth =3) # moyenne sur 3/4 jours par morceaux
            pyplot.ylim(0)
            (ymin, ymax) = pyplot.gca().yaxis.get_view_interval()
            pyplot.gca().xaxis.set_ticks([3,10,17])
         
            pyplot.text(25,0.8*ymax,'progression hebdo : ' + repr(round(100*(kE-1))) + ' %'  , color='black')
            pyplot.text(25,0.7*ymax,'progression finale : ' + repr(round(100*(kEfin-1)))+ ' %' , color='black')
            #pyplot.text(22,0.15*ymax,'divergence : ' + repr(divergence)   , color='black')
            pyplot.text(25,0.2*ymax,'Actuel : ' + repr(round(np.exp(regressor.predict([[njour-1+0]]))[0,0],1)) + ' Entree' 
                        , color='black')
            pyplot.text(25,0.1*ymax,'+ 2s : ' + repr(round(np.exp(regressor.predict([[njour-1+14]]))[0,0],1)) + ' Entree' 
                        , color='black')   
            pyplot.grid()
            pf = pf+(1)
        pyplot.show()  

#croissance_exp('France')

In [ ]:
# Graphique XY des taux d'hospitalisation et d'entrées à l'hôpital
def graphe_Thosp_Treprod(jour): 
    pyplot.figure(figsize = (15, 10))
    pyplot.xlim(0,150);pyplot.ylim(0,16)
    coord = np.zeros((150,2),dtype = float);j = 0
    i = Dj.loc[(Dj['DEP']=='France') & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
    pyplot.title(Dj.at[i-jour,'jour'])
    for DEP in Dj['DEP'].drop_duplicates():
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
        i = i - jour
        if DEP == 'France':col = 'red'; font = 12
        elif DEP=='Puy-de-Dôme':col ='blue' ; font = 12;bcol ='white'
        elif Dj.at[i,'PTOT'] >= 500000: col='black';font = 7;bcol ='white'
        else: col='black';font = 7;bcol =''
        pos = 0
        for k in range(j+1):
            if abs(Dj.at[i,'Thosp'] - coord[k,0]) <= 10 and Dj.at[i-3,'Treprodmoy'] == coord[k,1]:
                if pos == 0 : pos = 0.2
                elif pos == 0.2 :pos =-0.2
                elif pos == -0.2 :pos =0.5
                elif pos == 0.4 :pos = - 0.4
                elif pos == -0.4 :pos = 0 
        coord[j+1,0] = Dj.at[i,'Thosp'];coord[j+1,1] = Dj.at[i-3,'Treprodmoy'];j= j + 1
        if DEP == 'France':
            for s in range(0,5):
                pyplot.text(Dj.at[i-s*7,'Thosp'],min(16,Dj.at[i-3-s*7,'Treprodmoy']),('S-'+str(s)), 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font,backgroundcolor = 'white',fontweight = 'bold',zorder=4)
        elif Dj.at[i,'PTOT'] >= 500000:
            pyplot.text(Dj.at[i,'Thosp'],min(16,Dj.at[i-3,'Treprodmoy']+pos),DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font,fontweight = 'bold' ,zorder=3)
        else:
            pyplot.text(Dj.at[i,'Thosp'],min(16,Dj.at[i-3,'Treprodmoy']+pos),DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font ,zorder=2)
    pyplot.grid()
    axes = pyplot.gca()
    axes.add_artist(matplotlib.patches.Rectangle((60, 8), 120, 25, color = 'red',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((15, 16), 45, 25, color = 'red',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((60, 0), 120, 8, color = 'darkorange',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((15, 8), 45, 8, color = 'deeppink',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((0, 0), 15, 8, color = 'green',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((0, 8), 15, 25, color = 'hotpink',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((15, 0), 45, 8, color = 'yellow',alpha = 0.5))
    axes.xaxis.set_ticks(range(0,136,15))
    axes.yaxis.set_ticks(range(0,17,2))
    
    pyplot.xlabel ("Taux d'hospitalisation (pour 100 000 hab.)")
    pyplot.ylabel ("Taux d'entrée à l'hopital (pour 100 hosp.)")
#for jour in range(0,-1,-7):graphe_Thosp_Treprod(jour)


In [ ]:
# Graphique XY : entrées hospitalières / variations hebdomadaire
def graphe_Emoy_VarEmoy(jour): 
    pyplot.figure(figsize = (15, 10))
    pyplot.xlim(0,10);pyplot.ylim(-40,100)
    coord = np.zeros((150,2),dtype = float);j = 0
    i = Dj.loc[(Dj['DEP']=='France') & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
    pyplot.title(Dj.at[i-jour,'jour'])
    for DEP in Dj['DEP'].drop_duplicates():
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
        i = i - jour
        if DEP == 'France':col = 'red'; font = 10
        elif DEP=='Puy-de-Dôme':col ='blue' ; font = 12;bcol ='white'
        elif Dj.at[i,'PTOT'] < 200000: col='black';font = 7;bcol ='white'
        elif Dj.at[i,'PTOT'] >= 500000: col='black';font = 10;bcol ='white'
        else: col='black';font = 10;bcol =''
        if DEP == 'France':
            for s in range(0,3,1):
                pyplot.text(min(10,Dj.at[i-3-s*7,'Emoy']),
                    max(-40,min(100,pow(Dj.at[i-3-s*7,'Emoy']/Dj.at[i-17-s*7,'Emoy'],0.5)*100-100)),
                    ('s-'+str(s)), horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font,backgroundcolor = 'white',fontweight = 'bold',zorder=4)
                imax = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']=='2020-11-06')]['Emoy'].idxmax()
                pyplot.text(min(10,Dj.at[imax,'Emoy']),
                    max(-40,min(100,pow(Dj.at[imax,'Emoy']/Dj.at[imax-17,'Emoy'],0.5)*100-100)),
                    (Dj.at[imax,'jour']), horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font,backgroundcolor = 'white',fontweight = 'bold',zorder=1.8)
        elif Dj.at[i,'PTOT'] >= 500000:
            pyplot.text(min(10,Dj.at[i-3,'Emoy']),max(-40,min(100,pow(Dj.at[i-3,'Emoy']/Dj.at[i-17,'Emoy'],0.5)*100-100)),DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font,fontweight = 'bold' ,zorder=3)
        else:
            pyplot.text(min(10,Dj.at[i-3,'Emoy']),max(-40,min(100,pow(Dj.at[i-3,'Emoy']/Dj.at[i-17,'Emoy'],0.5)*100-100)),DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    rotation = 0,color = col,fontsize = font ,zorder=2)
    pyplot.grid()
    axes = pyplot.gca()
    
   
    axes.add_artist(matplotlib.patches.Rectangle((4, 0), 6, 100, color = 'red',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((8, -20), 2, 20, color = 'red',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((2, 30), 2, 70, color = 'darkorange',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((3, 0), 1, 30, color = 'darkorange',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((4, -20), 4, 20, color = 'darkorange',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((8, -40), 2, 20, color = 'darkorange',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((0, 30), 2, 70, color = 'yellow',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((2, 0), 1, 30, color = 'yellow',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((3, -20), 1, 20, color = 'yellow',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((4, -40), 4, 20, color = 'yellow',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((0, -40), 2, 70, color = 'green',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((2, -40), 1, 40, color = 'green',alpha = 0.5))
    axes.add_artist(matplotlib.patches.Rectangle((3, -40), 1, 20, color = 'green',alpha = 0.5))
    
    axes.xaxis.set_ticks(range(0,10,1))
    axes.yaxis.set_ticks(range(-40,101,10))
    
    pyplot.xlabel ("Entrées à l'hôpital (pour 100 000 hab.)")
    pyplot.ylabel ("Variation hebdo des entrées à l'hôpital (%)")
    
#graphe_Emoy_VarEmoy(0)

In [ ]:
# liste des alertes sur les entrées hospitalières

lDEPcroissant = [] # 
ltexte = []

texte =  ('Departement'.ljust(25)+'Hosp'.rjust(5)+' | '+'Treprod'.rjust(7)+' | '
          +'S-2'.rjust(4)+' | '+' S-1'.rjust(4)+' | '+'Entree'.rjust(6)+' | '+'VarE'.rjust(5)+' | '
         +'Alerte'.ljust(20)+' | ')

ltexte.append(['',texte])

lDEP = Dj.loc[Dj['jour'] == Dj['jour'].max(),['DEP','Thosp']].sort_values(
      by ='Thosp',ascending = False)['DEP'].drop_duplicates()

for DEP in lDEP:
    i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
   
    Emoy3 = Dj.at[i-3,'Emoy']
    Emoy10 = Dj.at[i-10,'Emoy']
    Emoy17 = Dj.at[i-17,'Emoy']
    if Emoy17 > 0:
        VEmoy = (pow(Emoy3 / Emoy17,0.5)-1) * 100
     
    alerte = ''
    if  Emoy3 >= 4 and VEmoy >= 0:alerte = 'critique'
    elif Emoy3 >= 8 and VEmoy >= -20:alerte = 'critique'
    elif Emoy3 >= 2 and VEmoy >= 30:alerte = 'Proche critique'
    elif Emoy3 >= 3 and VEmoy >= 0:alerte = 'Proche critique'
    elif Emoy3 >= 4 and VEmoy >= -20:alerte = 'Fin critique'
        
    if alerte != '' or DEP =='France':
        lDEPcroissant.append(DEP)
            
        texte = (DEP.ljust(25) +
                  str(round(Dj.at[i,'Thosp'])).rjust(3)+'%'.rjust(2)+' | '+
                  str(round(Dj.at[i-3,'Treprodmoy'],1)).rjust(5)+'%'.rjust(2)+' | '+
                  str(round(Emoy17,1)).rjust(4)+' | ' + 
                  str(round(Emoy10,1)).rjust(4)+' | ' + 
                  str(round(Emoy3,1)).rjust(6)+' | ' +
                  str(int(VEmoy)).rjust(3)+'%'.rjust(2)+' | '+
                  alerte.ljust(20)+' | ')
        ltexte.append([DEP,texte])

#print('Alerte sur les entrées hospitalières')
'''
print()
if len(lDEPcroissant)> 0:
    for ligne in ltexte: print(ligne[1])
    courbe_departement(lDEPcroissant)
else:print('RAS')
'''
print()

In [ ]:
def graphe_date_Emax(): # date des entrées max au 2ème confinement
    pyplot.figure(figsize = (15, 10))
    ifin = Dj.loc[(Dj['DEP']=='France') & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
    idebut = Dj.loc[(Dj['DEP']=='France') & (Dj['jour']=='2020-10-17')]['Emoy'].idxmax()
    pyplot.ylim(0,20)
    pyplot.plot(Dj['jour'].iloc[idebut:ifin],Dj['Emoy'].iloc[idebut:ifin],color='blue') # pour inscrire l'axe des x (date)
    for DEP in Dj['DEP'].drop_duplicates():
        i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']>= '2020-10-17')]['Emoy'].idxmax()
        jourmax = Dj.at[i,'jour']
        Emax  = Dj.at[i,'Emoy']
        if DEP == 'France':
            pyplot.text(jourmax,Emax,DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    color = 'blue',fontsize = 12,fontweight = 'bold' ,zorder=4)
        elif Dj.at[i,'PTOT'] >= 500000:
            pyplot.text(jourmax,Emax,DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    color = 'black',fontsize = 7,fontweight = 'bold' ,zorder=3) 
        else:
            pyplot.text(jourmax,Emax,DEP, 
                    horizontalalignment = 'left', verticalalignment = 'center',
                    color = 'black',fontsize = 7 ,zorder=2) 
    for label in pyplot.gca().xaxis.get_ticklabels():
        label.set_rotation(90)
    pyplot.title("Date où les entrées à l'hôpital sont maximales")
#graphe_date_Emax()

## IV. Décomposition des entrées hospitalières (ACP)

In [ ]:
Nbjour = datetime.timedelta(days = 24); date24 = datemax - Nbjour;Sdate24 = str(date24)
Nbjour = datetime.timedelta(days = 3); date3 = datemax - Nbjour;Sdate3 = str(date3)

In [ ]:
# Les variables à expliquer sont les taux d'hospitalisation quotidients (variable jour) 
#  associés à chaque département (individu)
X = Dj.loc[(Dj['jour'] >= Sdate24) & (Dj['jour'] <= Sdate3) ,['DEP','jour','Emoy']]
X = X.pivot(index = 'DEP', columns = 'jour', values = 'Emoy')

In [ ]:
pca = decomposition.PCA() #création de l'analyse
pca.n_components = 5 # Choix de 2 composantes (en dessous de 1, indique le % de variances à expliquer)
pca.fit(X)
# ratio de la variance expliquée (visualisation graphique de chaque composante)
pyplot.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_ * 100,color="cyan")
pyplot.xlabel('Numéro de composante' , fontsize = 12) # titre des absisses
pyplot.ylabel('% impact' , fontsize = 12) # titre des absisses
for i in range(len(pca.explained_variance_ratio_)):
    pyplot.text(i, pca.explained_variance_ratio_ [i]* 100/2, round(pca.explained_variance_ratio_ [i]* 100,2),
                horizontalalignment = 'center',fontweight = 'bold')

In [ ]:
pca = decomposition.PCA() #création de l'analyse définitive
pca.n_components = 2 # Choix de 2 composantes (en dessous de 1, indique le % de variances à expliquer)
pca.fit(X)

In [ ]:
# affichage des coordonnées des axes principaux
# cela montre l'impact de chaque composante par rapport à la moyenne du jour (en non en absolue)
pyplot.figure(5,figsize = (15, 5))
pyplot.subplot(1,2,1)
if pca.n_components > 0 :pyplot.plot(pca.components_[0,:],label='Pca1')
if pca.n_components > 1 :pyplot.plot(pca.components_[1,:],label='Pca2')
if pca.n_components > 2 :pyplot.plot(pca.components_[2,:],label='Pca2')
pyplot.grid()
pyplot.legend()
pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
pyplot.subplot(1,2,2)
pyplot.plot(X.mean(),label ='moyenne du jour',color='black')
pyplot.grid()
pyplot.legend()
pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
pyplot.ylabel('Entrées / 100 000 hab.' , fontsize = 12) # titre des absisses
pyplot.ylim(0,10)
for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
    label.set_rotation(90)

In [ ]:
#pca.transform(X) # les coordonnées des points transformés sur les axes principaux (un point par ligne).
# Ca donne l'impact de chaque composante sur le département
# remarques ces axes principaux sont centrés sur les moyennes de chaque journée

#positionnement des individus dans le premier plan
fig, axes = pyplot.subplots(figsize=(15,5))
axes.set_xlim(pca.transform(X)[:,0].min(),pca.transform(X)[:,0].max()) #même limites en abscisse
axes.set_ylim(pca.transform(X)[:,1].min(),pca.transform(X)[:,1].max()) #et en ordonnée
#placement des étiquettes des observations
for i in range(len(X)):
    col ='grey'
    police = 'normal'
    if  X.index[i] == 'France': col = 'black';police = 'bold'
    if  X.index[i] == 'Puy-de-Dôme': col = 'blue';police = 'bold'
    if Dj.loc[(Dj['DEP'] == X.index[i]) & (Dj['jour'] == Dj['jour'].max())]['incoherence'].any() == True: col = 'red'
    pyplot.annotate(X.index[i],(pca.transform(X)[i,0],pca.transform(X)[i,1]),color=col,fontweight = police)
#ajouter les axes
pyplot.plot([pca.transform(X)[:,0].min(),pca.transform(X)[:,0].max()],[0,0],color='silver',linestyle='-',linewidth=1)
pyplot.plot([0,0],[pca.transform(X)[:,1].min(),pca.transform(X)[:,1].max()],color='silver',linestyle='-',linewidth=1)
pyplot.xlabel('Evolution par rapport à la moyenne nationale' , fontsize = 12) # titre des absisses
pyplot.ylabel('Décroissance' , fontsize = 12) # titre des absisses
#affichage
pyplot.show()

In [ ]:
ldep = ['France','Lozère','Jura','Puy-de-Dôme','Meuse','Vienne','Landes']
lcol = ['black','red','deeppink','blue','violet','orange','limegreen']
fig6 = pyplot.figure(figsize = (10, 5))
for dep in range(len(ldep)):
       # if Dj.at[i,'jour'] == Dj['jour'].max() : # recherche de la dernière l'igne du département
        # Choix des couleurs, épaisseur en fonction du taux hosp, taux reprod et tendance
    epaisseur = 3;legende ='';col ='grey'
    # Courbe du département       
    Djdep = Dj.loc[(Dj['DEP'] == ldep[dep]) & (Dj['jour'] >= '2020-12-01'),['jour','Thosp']] # récupération du département
    pyplot.plot(Djdep['jour'],Djdep['Thosp'],lcol[dep],linewidth = epaisseur,label = ldep[dep])
    # Paramètres généraux
pyplot.legend(loc='upper left')
pyplot.title("Evolution des hospitalisations par catégorie de département")
pyplot.xlabel('Date' , fontsize = 12) # titre des absisses
pyplot.ylabel('hosp. / 100 000 hab. ' , fontsize = 12) # titre des ordonnées
for label in pyplot.gca().xaxis.get_ticklabels(): # mise en forme du label des x (rotation verticale)
    label.set_rotation(90)
pyplot.grid() # grille
pyplot.show()  
#fig6.savefig('Images\Evolution des hospitalisations par catégorie de département.png')

## V. Surveillance des hospitalisations et des tests de dépistage

### A - Taux d'hospitalisation

In [ ]:
courbe_TxHosp('France')

### B - Taux d'entrée à l'hôpital

In [ ]:
courbe_TxEntree('France')

### C - Taux de tests virologiques positifs

In [ ]:
courbe_TxTestPositif()

### D - Alertes sur les entrées à l'hôpital

In [ ]:
#carte_alerte() (à mettre ici une fois remontée l'alerte PLSR sur les entrées hosp.)

## VI.Analyse détaillée d'un département

In [ ]:
# Création d'un widget pour saisir un département 
def Synthese_departement(departement):
    courbe_departement([departement]);croissance_exp(departement)
    return Dj.loc[Dj['DEP'] == departement,
                  ['jour','hosp','dc','Entree','Emoy','Sortie','pmoy','Thosp','CThosp',
                   'Treprod','Treprodmoy','CTreprod','Dureehosp','alerte','alerteconf']].iloc[-21:]
interact(Synthese_departement, departement = widgets.Dropdown(
        options= Dj[Dj['jour'] == Dj['jour'].max()]['DEP'],value='Puy-de-Dôme'))

In [ ]:
Synthese_departement("France")


## IV.Régression des entrées hospitalières (PLS)

In [ ]:
## régression PLS des entrées hospitalières sur les tests positifs

## Structuration des données
npoint =3 # nombre de variables explicatifs (tests positifs)
nobs = 1 # nombre de variables à observer (entrées hospitalières)
YET = np.zeros((len(Dj), 1), dtype = float)
XET = np.zeros((len(Dj), npoint), dtype = float)
IndexET = np.zeros((len(Dj),1), dtype = int)
j = 0
for i in range(len(Dj)): # Parcours des lignes dans l'ordre département + jour
    if  Dj.at[i,'jour'] >= '2020-09-14' : #and Dj.at[i,'jour'] <= '2020-12-27' :
            if (i > 24 and i < len(Dj)-3 and Dj.at[i-24,'dep'] == Dj.at[i,'dep'] 
                and Dj.at[i+3,'dep'] == Dj.at[i,'dep']
                and Dj.at[i,'Emoy'] <=10
                and Dj.at[i-10,'Emoy'] <=10
                and Dj.at[i,'Emoy'] >= 0.5
                and Dj.at[i,'Emoy'] * Dj.at[i,'PTOT'] / 100000 >= 3
                and datetime.date.fromisoformat(Dj.at[i,'jour']).isoweekday() == 2
               ):
                YET[j] = Dj.at[i,'Emoy']
                
                XET[j,0] = Dj.at[i-13,'pmoy']
                XET[j,1] = Dj.at[i-20,'pmoy']
                #XET[j,2] = Dj.at[i-27,'pmoy']
                XET[j,2] = Dj.at[i-10,'Emoy']
                #XET[j,5] = Dj.at[i-24,'Emoy']
                IndexET[j] = i # pour retrouver la ligne d'origine
                j = j + 1
# Redimensionne                
XET = XET[0:j,:]
IndexET = IndexET[0:j]
YET = YET[0:j]
NET = j
# Centrage / réduction
YETm =YET.mean(0)
YETe = YET.std(0)
YETcr = (YET - YETm)/YETe # observation centrée réduite
XETm = XET.mean(0) # moyenne par variables
XETe = XET.std(0)
XETcr = (XET - XETm)/XETe # variables explicatives centrées
print('X Moy :',np.round(XETm,1),'std :',np.round(XETe,1))
print('Y Moy :',np.round(YETm,1),'std :',np.round(YETe,1))
print('Nb point :',NET)

In [ ]:
# Impact des composantes et validation croisée => choisir le nombre de composante

RESSETn = YETcr.var(0)
Xn = XETcr
Yn = YETcr
pls = PLSRegression(n_components=1,scale=True)

for n in range(1,npoint+1):
    plsETn= PLSRegression(n_components=n,scale=True)
    plsETn.fit(XETcr, YETcr)
    PREn = ((plsETn.predict(XETcr) - YETcr)*YETe).std(0,ddof = 1) # erreur standard
    # R2 sur l'ensemble de validation pour vérifier l'absence de sur-apprentissage
    scoreET = cross_val_score(plsETn, XETcr,YETcr, cv = 5,scoring ='r2') 
    # itération croisée à l'étape n 
    Yn_cvp= cross_val_predict(pls, Xn,Yn, cv = 5)
    PRESSn = (Yn_cvp - Yn).var(0) # somme des erreurs quadratique sur les valeurs prédites à l'étape n
    Q = 1 - PRESSn / RESSETn # traduit l'amélioration sur l'erreur de la nouvelle étape ()
    # iteration globale
    pls.fit(Xn, Yn)
    Yn_p = pls.predict(Xn)
    RESSETn = (Yn_p - Yn).var(0) # somme des erreurs quadratiques de l'étape n 
    Xn = Xn - np.dot(pls.x_scores_[:,0].reshape(-1,1),pls.x_loadings_[:,0].reshape(1,-1))
    Yn = Yn - np.dot(pls.x_scores_[:,0].reshape(-1,1),pls.y_loadings_[:,0].reshape(1,-1))
    
    print('Etape :',n,'R2 >= 0.7 :',str(round(plsETn.score(XETcr,YETcr),2)).ljust(6)
           ,'Moy R2 val. :',str(round(scoreET.mean(),2)).ljust(6)
           ,'Disp R2 val. :',str(round(scoreET.std(),2)).ljust(6)
           , "Val. croisée Q2 > 0.05 :", str(np.round(Q,3)[0]).rjust(6)
           , "Erreur", str(np.round(PREn,2)[0]).rjust(6))

In [ ]:
# Regression PLS et performance
ncpET = 2
plsET = PLSRegression(n_components=ncpET ,scale=True)
plsET.fit(XETcr, YETcr)
print('R2 :'.ljust(35),round(plsET.score(XETcr,YETcr),2))
PREn = ((plsETn.predict(XETcr) - YETcr)*YETe).std(0,ddof = 1) # erreur standard
print('Erreur standard :'.ljust(35),np.round(PREn,2)[0],'Entrées pour 100 000 hab.')
print("Moyenne de l'écart relatif :".ljust(35),int((abs(plsET.predict(XETcr)-YETcr)*YETe/(YET)*100).mean()),'%')
print("Dispersion de l'écart relatif :".ljust(35),int((abs(plsET.predict(XETcr)-YETcr)*YETe/(YET)*100).std()),'%')
# analyse des écarts entre prédiction / réel

vrai_predit = 0
vrai_nonpredit = 0
faux_vu_vrai = 0
for i in range(len(YET)):
   
    if plsET.predict(XETcr[i,:].reshape(1, -1))*YETe+YETm >= 5:
        if YET[i] >= 4 : vrai_predit = vrai_predit + 1
        else: faux_vu_vrai = faux_vu_vrai + 1
    elif plsET.predict(XETcr[i,:].reshape(1, -1))*YETe+YETm < 3:
        if YET[i] >= 4 : vrai_nonpredit = vrai_nonpredit + 1

print('Alertes pertinentes'.ljust(35), str(vrai_predit).rjust(4),'(prédit > 5 et réellement  > 4)')
print('Fausses alertes'.ljust(35),str(faux_vu_vrai).rjust(4),'(prédit > 5 alors que < 4)')
print('cas non détectés'.ljust(35),str(vrai_nonpredit).rjust(4) ,'(prédit < 3 alors que réellement > 4)')

In [ ]:
# analyse des écarts entre prédiction / réel
pyplot.figure(figsize =[15,5])
pyplot.subplot(1,3,1)
pyplot.plot(YET,plsET.predict(XETcr)*YETe+YETm,linestyle= 'none', marker ='.',color='red',markersize =2)
pyplot.plot(YET,YET,linestyle= 'none', marker ='.',color='green')
pyplot.ylim(0,12)
pyplot.xlim(0,12)
pyplot.xlabel("Observation")
pyplot.ylabel("Estimation")
pyplot.grid()
pyplot.subplot(1,3,2)
pyplot.plot(YETcr,(plsET.predict(XETcr)-YETcr),linestyle= 'none', marker ='.',color='blue',markersize =2)
pyplot.xlabel("Observation centrée réduite")
pyplot.ylabel("Erreur réduite")
pyplot.grid()
pyplot.subplot(1,3,3)
pyplot.plot(XET[:,2],(plsET.predict(XETcr)-YETcr)*YETe,linestyle= 'none', marker ='.',color='blue',markersize =2)
pyplot.xlabel("Entrée")
pyplot.ylabel("Erreur")
pyplot.grid()
pyplot.show()

In [ ]:
# interpréation des composantes
pyplot.figure(figsize =[10,5])
pyplot.subplot(1,2,1)
pyplot.plot(plsET.x_loadings_[:,0],color ='blue',label = 'P1',linestyle='none',marker='o')
if ncpET >=2: pyplot.plot(plsET.x_loadings_[:,1],color ='orange',label ='P2',linestyle='none',marker='o')
if ncpET >=3:pyplot.plot(plsET.x_loadings_[:,2],color ='red',label ='P3',linestyle='none',marker='o')
pyplot.grid()
pyplot.legend()
pyplot.subplot(1,2,2)
pyplot.plot(plsET.y_loadings_[:,0],color ='blue',label = 'Q1',linestyle='none',marker='x')
if ncpET >=2:pyplot.plot(plsET.y_loadings_[:,1],color ='orange',label ='Q2',linestyle='none',marker='x')
if ncpET >=3:pyplot.plot(plsET.y_loadings_[:,2],color ='red',label ='Q3',linestyle='none',marker='x')
pyplot.grid()
pyplot.legend()
pyplot.show()

In [ ]:
# reconsitution à partir des composantes
p = 1000
#pyplot.ylim(0,100)
pyplot.plot(XETm[:],color= 'black',label='moyenne')
pyplot.plot(XETm[:]+plsET.x_scores_[p,0]*plsET.x_loadings_[:,0]*XETe,color= 'blue',label ='CP1')
if ncpET >=2:pyplot.plot(XETm[:]+plsET.x_scores_[p,0]*plsET.x_loadings_[:,0]*XETe
            +plsET.x_scores_[p,1]*plsET.x_loadings_[:,1]*XETe,color= 'orange',label ='CP1/2')
if ncpET >=3:pyplot.plot(XETm[:]+plsET.x_scores_[p,0]*plsET.x_loadings_[:,0]*XETe
            +plsET.x_scores_[p,1]*plsET.x_loadings_[:,1]*XETe
            +plsET.x_scores_[p,2]*plsET.x_loadings_[:,2]*XETe,color= 'red',label ='CP1/2/3')
pyplot.plot(XET[p,:],'green',label='reel')
pyplot.title("Décomposition d'un exemple")
pyplot.legend()
pyplot.grid()

In [ ]:
# correlation des scores t,u pour chaque composante => pour retenir ou non une composante
pyplot.figure(figsize =[15,5])
pyplot.subplot(1,3,1)
pyplot.title('correlation t/u pour la composante 1')
pyplot.plot(plsET.x_scores_[:,0],plsET.y_scores_[:,0],linestyle= 'none', marker ='.',color='blue',markersize=2)
pyplot.plot(plsET.x_scores_[:,0],plsET.x_scores_[:,0],linestyle= 'none', marker ='.',color='green',markersize=2)
pyplot.grid()

if ncpET >=2:
    pyplot.subplot(1,3,2)
    pyplot.title('correlation t/u pour la composante 2')
    pyplot.plot(plsET.x_scores_[:,1],plsET.y_scores_[:,1],linestyle= 'none', marker ='.',color='blue',markersize=2)
    pyplot.plot(plsET.x_scores_[:,1],plsET.x_scores_[:,1],linestyle= 'none', marker ='.',color='green',markersize=2)
    pyplot.grid()
    
if ncpET >=3:
    pyplot.subplot(1,3,3)
    pyplot.title('correlation t/u pour la composante 3')
    pyplot.plot(plsET.x_scores_[:,2],plsET.y_scores_[:,2],linestyle= 'none', marker ='.',color='blue',markersize=2)
    pyplot.plot(plsET.x_scores_[:,2],plsET.x_scores_[:,2],linestyle= 'none', marker ='.',color='green',markersize=2)
    pyplot.grid()
pyplot.show()

In [ ]:
# Paramètres du modèle
print("matrice des charges/poids Q tel que Y=TQ'+ e ",
      np.round(plsET.y_loadings_[:,:],2),np.round(plsET.y_weights_[:,:],2))
print("Matrice des poids W* (T=XW*) dans l'espace de départ") 
print(np.round(plsET.x_rotations_[:,:],2))
print("Matrice du modèle linéaire centré réduit Y = X * B + e où B=W*Q':",np.round(plsET.coef_[:,:].transpose(),2))
print("Dans l'espace d'origine:",np.round(plsET.coef_[:,:].transpose()/XETe*YETe,3))
print("Constante : ",np.round((YETm/YETe- plsET.coef_[2]*XETm[2]/XETe[2])*YETe,2))
# Carte des charges 1 et 2
pyplot.figure(figsize =[5,5])
pyplot.title('Carte des variables : w*c1/w*c2')
pyplot.xlim(-1,1)
pyplot.ylim(-1,1)
pyplot.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
pyplot.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
for i in range(0,npoint):
    pyplot.annotate('X'+str(i),(plsET.x_rotations_[i,0],plsET.x_rotations_[i,1]),color='black',fontweight = 'normal')
    pyplot.plot([0,plsET.x_rotations_[i,0]],[0,plsET.x_rotations_[i,1]],color='black',linestyle='--')
pyplot.annotate('Y',(plsET.y_loadings_[0,0],plsET.y_loadings_[0,1]),color='red',fontweight = 'normal')
pyplot.plot([0,plsET.y_loadings_[0,0]],[0,plsET.y_loadings_[0,1]],color='red',linestyle='--')
axes=pyplot.gca()
axes.add_artist(matplotlib.patches.Circle((0,0), 1.0, edgecolor = 'black',facecolor = 'none' ))
pyplot.show()
# Carte des charges 1 et 3
if ncpET >=3:
    pyplot.figure(figsize =[5,5])
    pyplot.title('Carte des variables : w*c2/w*c3')
    pyplot.xlim(-1,1)
    pyplot.ylim(-1,1)
    pyplot.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
    pyplot.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
    for i in range(0,npoint):
        pyplot.annotate('X'+str(i),(plsET.x_rotations_[i,1],plsET.x_rotations_[i,2]),color='black',fontweight = 'normal')
        pyplot.plot([0,plsET.x_rotations_[i,1]],[0,plsET.x_rotations_[i,2]],color='black',linestyle='--')
    pyplot.annotate('Y',(plsET.y_loadings_[0,1],plsET.y_loadings_[0,2]),color='red',fontweight = 'normal')
    pyplot.plot([0,plsET.y_loadings_[0,1]],[0,plsET.y_loadings_[0,2]],color='red',linestyle='--')
    axes=pyplot.gca()
    axes.add_artist(matplotlib.patches.Circle((0,0), 1.0, edgecolor = 'black',facecolor = 'none' ))
    pyplot.show()
    
    pyplot.figure(figsize =[5,5])
    pyplot.title('Carte des variables : w*c1/w*c3')
    pyplot.xlim(-1,1)
    pyplot.ylim(-1,1)
    pyplot.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
    pyplot.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
    for i in range(0,npoint):
        pyplot.annotate('X'+str(i),(plsET.x_rotations_[i,0],plsET.x_rotations_[i,2]),color='black',fontweight = 'normal')
        pyplot.plot([0,plsET.x_rotations_[i,0]],[0,plsET.x_rotations_[i,2]],color='black',linestyle='--')
    pyplot.annotate('Y',(plsET.y_loadings_[0,0],plsET.y_loadings_[0,2]),color='red',fontweight = 'normal')
    pyplot.plot([0,plsET.y_loadings_[0,0]],[0,plsET.y_loadings_[0,2]],color='red',linestyle='--')
    axes=pyplot.gca()
    axes.add_artist(matplotlib.patches.Circle((0,0), 1.0, edgecolor = 'black',facecolor = 'none' ))
    pyplot.show()

In [ ]:
# Recherches des alertes actuelles sur entrées hospitalières et tests positifs
X = np.zeros((1, npoint), dtype = float)
for DEP in Dj['DEP'].drop_duplicates():
    i = Dj.loc[(Dj['DEP']==DEP) & (Dj['jour']==Dj['jour'].max())]['Emoy'].idxmax()
    X[0,0] = Dj.at[i-6,'pmoy']
    X[0,1] = Dj.at[i-13,'pmoy']
    X[0,2] = Dj.at[i-3,'Emoy']
    #X[0,3] = Dj.at[i-10,'Emoy']
    Xcr = (X - XETm) / XETe
    Yp = np.round((plsET.predict(Xcr)*YETe+YETm)[0,0],1) # prédit dans 1 semaine
    Ya = round(Dj.at[i-3,'Emoy'],1) # actuel
    alerte = ''
    Dj.at[i,'alertePLSR'] = 0
    if Ya >= 2.8 or Yp >= 3.8  or Dj.at[i,'DEP'] =='France': 
        if Ya < 2.8  and Yp >= 5 : alerte = ' => critique';Dj.at[i,'alertePLSR'] = 2
        elif Ya < 2.8 and Yp >= 3.8  : alerte = '=> alerte';Dj.at[i,'alertePLSR'] = 1
        elif Ya >= 2.8 and Ya < 4 and Yp >= 5  : alerte = ' => critique';Dj.at[i,'alertePLSR'] = 2
        elif Ya >= 2.8 and Ya < 4 and Yp >= 1.8  : alerte = ' = alerte';Dj.at[i,'alertePLSR'] = 1
        elif Ya >= 2.8 and Ya < 4 and Yp < 1.8  : alerte = 'fin alerte';Dj.at[i,'alertePLSR'] = 0
        elif Ya >= 2.8 and Yp < 4 : alerte = '= alerte';Dj.at[i,'alertePLSR'] = 1
        elif Ya >= 4 and Yp < 1.8 : alerte = 'fin alerte' ;Dj.at[i,'alertePLSR'] = 0
        elif Ya >= 4 and Yp < 3 : alerte = 'retour alerte' ;Dj.at[i,'alertePLSR'] = 1
        elif Ya >= 4 and Yp >= 3 : alerte = '= critique';Dj.at[i,'alertePLSR'] = 2
        print(Dj.at[i,'DEP'].ljust(25),': Hosp',str(Dj.at[i,'Thosp']).ljust(3),'%'
              ,': PTOT',str(int(Dj.at[i,'PTOT'])).rjust(8)
              ,'| S0',str(Ya).rjust(4),'| S+1',str(Yp).rjust(4),'|',alerte.ljust(20))

In [ ]:
carte_alerte()

In [ ]:
'''
# Envoi fichier sur github pour alimenter la réutilisation data gouv
###('https://github.com/smarcovici/Covid_19/tree/master/Surveillance_deconfinement/Images')
import ftplib # on importe le module et on la renomme juste pour le script en "ftp

host = "github.com" # adresse du serveur FTP
user = "smarcovici" # votre identifiant
password = "19Compte75" # votre mot de passe
monport = 3636
ftp = ftplib.FTP('')
ftp.connect(host,monport)
ftp.login(user,password)

fichier = 'Images\Alertes_entrees_' + Dj['jour'].max() + '.png'
#fig.savefig('Images\Alertes sur les entrees_' + Dj['jour'].max() + '.png')
file = open(fichier, 'rb') # ici, j'ouvre le fichier ftp.py 
ftp.storbinary('STOR fichier', file) # ici (où connect est encore la variable de la connexion), j'indique le fichier à envoyer
file.close() # on ferme le fichier
connect.quit() # où "connect" est le nom de la variable dans laquelle vous avez déclaré la connexion 

'''
print()

In [4]:
!mise_a_jour_remote.cmd


c:\Users\sylvain_2\Documents\Python Scripts\Beyond_basics\Covid_19\Covid_19\Surveillance_deconfinement>cd C:\Users\sylvain_2\Documents\Python Scripts\Beyond_basics\Covid_19\Covid_19\Surveillance_deconfinement\Images 

C:\Users\sylvain_2\Documents\Python Scripts\Beyond_basics\Covid_19\Covid_19\Surveillance_deconfinement\Images>git add . 

C:\Users\sylvain_2\Documents\Python Scripts\Beyond_basics\Covid_19\Covid_19\Surveillance_deconfinement\Images>git commit -m "nouvelle version" 
[master bd68fce] nouvelle version
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Alertes_entrees_recentes - Copie.png

C:\Users\sylvain_2\Documents\Python Scripts\Beyond_basics\Covid_19\Covid_19\Surveillance_deconfinement\Images>git push -u origin master 
Branch 'master' set up to track remote branch 'master' from 'origin'.


To https://github.com/smarcovici/visualisation_covid_19.git
   77ecf2d..bd68fce  master -> master


In [5]:
!git

usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone             Clone a repository into a new directory
   init              Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add               Add file contents to the index
   mv                Move or rename a file, a directory, or a symlink
   restore           Restore working tree files
   rm                Remove files from the working tree and from the index
   sparse-checkout   Initialize and modify the sparse-checkout

examine the history and state (see also: git help revisio